In [ ]:
import numpy as np
import os
import numpy.ma as ma
import copy 
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
print(tf.__version__)
tf.executing_eagerly()


2.3.0


True

In [ ]:
class Representation:
  def __init__(self):

    temp = np.zeros((96,8,8))
    for i in range(8):
      temp[i,6,i]=1
      temp[16+i,1,i]=1
      if i%2==0:
        j=i
        temp[8+j,7,j//2],temp[8+j+1,7,7-j//2]=1,1
        temp[24+j,0,j//2],temp[24+j+1,0,7-j//2]=1,1
    
    self.player = np.ones((1,8,8))

    self.start_state = np.concatenate((temp,self.player))
        
    self.total_actions = 1258
    self.actions = np.concatenate(np.array([np.ones(16),np.zeros(88),np.zeros(280),np.ones(1),np.zeros(6),np.ones(2),np.zeros(6),np.ones(1),np.zeros(858)]))

    self.w_left_castle = True
    self.w_right_castle = True
    self.b_left_castle = True
    self.b_right_castle = True
    self.b_pawn_en_passant = []
    self.w_pawn_en_passant = []

  def apply_action(self,in_state,action):
    #print('player app act',in_state[96,0,0],'action->',action)
    state = copy.deepcopy(in_state)

    if state[96,0,0]==1:
      if action<104:
        plane = action//13
        pos=list(zip(*np.where(state[plane]==1)))[0]
        val = action%13 
        row,col = pos[0],pos[1]
        if val==0:
          row -= 1
          if row == 0:
            state[plane+32,row,col]=1
        elif val==1:
          row -= 2
          self.w_pawn_en_passant.append(plane)
        elif val==2:
          row,col = row-1,col-1
          if row == 2 and all([state[q,row,col]==0 for q in range(96)]):
            state[:,row+1,col]=0
          if row == 0:
            state[plane+32,row,col]=1
        elif val==3:
          row,col = row-1,col+1
          if row == 2 and all([state[q,row,col]==0 for q in range(96)]):
            state[:,row+1,col]=0
          if row == 0:
            state[plane+32,row,col]=1
        elif val==4:
          row -= 1
          state[plane+40,row,col]=1
        elif val==5:
          row -= 1
          state[plane+48,row,col]=1
        elif val==6:
          row -= 1
          state[plane+56,row,col]=1
        elif val==7:
          row,col = row-1,col-1
          state[plane+40,row,col]=1
        elif val==8:
          row,col = row-1,col-1
          state[plane+48,row,col]=1
        elif val==9:
          row,col = row-1,col-1
          state[plane+56,row,col]=1
        elif val==10:
          row,col = row-1,col+1
          state[plane+40,row,col]=1
        elif val==11:
          row,col = row-1,col+1
          state[plane+48,row,col]=1
        else:
          row,col = row-1,col+1
          state[plane+56,row,col]=1
      elif action<384:
        plane = 8 + (action-104)//28
        if plane>9:
          plane = (plane-9)+39
        pos=list(zip(*np.where(state[plane]==1)))[0]
        if plane == 8:
          self.w_left_castle = False
        elif plane == 9:
          self.w_right_castle = False
        act = ((action-104)%28)//7
        val = (action-104)%7 + 1
        row,col = pos[0],pos[1]
        if act==0:
          row -= val
        elif act==1:
          row += val
        elif act==2:
          col -= val
        else:
          col += val
      elif action<464:
        plane = 10+(action-384)//8
        if plane>11:
          plane = (plane-11)+47
        pos=list(zip(*np.where(state[plane]==1)))[0]
        val = (action-384)%8
        row,col = pos[0],pos[1]
        if val==0:
          row-=2
          col-=1
        elif val==1:
          row-=1
          col-=2
        elif val==2:
          row+=1
          col-=2
        elif val==3:
          row+=2
          col-=1
        elif val==4:
          row+=2
          col+=1
        elif val==5:
          row+=1
          col+=2
        elif val==6:
          row-=1
          col+=2
        else:
          row-=2
          col+=1
      elif action<744:
        plane = 12+(action-464)//28
        if plane>13:
          plane = (plane-13)+55
        pos=list(zip(*np.where(state[plane]==1)))[0]
        act = ((action-464)%28)//7
        val = (action-464)%7 + 1
        row,col = pos[0],pos[1]
        if act==0:
          row,col = row-val,col-val
        elif act==1:
          row,col = row-val,col+val
        elif act==2:
          row,col = row+val,col-val
        else:
          row,col = row+val,col+val
      elif action<1248:
        plane = 14+(action-744)//56
        if plane>14:
          plane = (plane-14)+31
        pos=list(zip(*np.where(state[plane]==1)))[0]
        act = ((action-744)%56)//7
        val = (action-744)%7 + 1
        row,col = pos[0],pos[1]
        if act==0:
          row -= val
        elif act==1:
          row += val
        elif act==2:
          col -= val
        elif act==3:
          col += val
        elif act==4:
          row,col = row-val,col-val
        elif act==5:
          row,col = row-val,col+val
        elif act==6:
          row,col = row+val,col-val
        else:
          row,col = row+val,col+val
      else:
        plane = 15
        self.w_left_castle=False
        self.w_right_castle=False
        pos=list(zip(*np.where(state[plane]==1)))[0]
        val = (action-1248)
        row,col = pos[0],pos[1]
        if val==0:
          row -= 1
        elif val==1:
          row,col = row-1,col-1
        elif val==2:
          col -= 1
        elif val==3:
          row,col = row+1,col-1
        elif val==4:
          row += 1
        elif val==5:
          row,col = row+1,col+1
        elif val==6:
          col += 1
        elif val==7:
          row,col = row-1,col+1
        elif val==8:
          col -= 2
          state[8,7,0]=0
          state[8,7,3]=1
        else:
          col += 2
          state[9,7,7]=0
          state[9,7,5]=1
      self.b_pawn_en_passant = []
    else:
      
      if action<104:
        plane = 16+action//13
        pos=list(zip(*np.where(state[plane]==1)))[0]
        val = action%13
        row,col = pos[0],pos[1]
        if val==0:
          row += 1
          if row == 7:
            state[plane+48,row,col]=1
        elif val==1:
          row += 2
          self.b_pawn_en_passant.append(plane)
        elif val==2:
          row,col = row+1,col+1
          if row == 5 and all([state[q,row,col]==0 for q in range(96)]):
            state[:,row-1,col]=0
          if row == 7:
            state[plane+48,row,col]=1
        elif val==3:
          row,col = row+1,col-1
          if row == 5 and all([state[q,row,col]==0 for q in range(96)]):
            state[:,row-1,col]=0
          if row == 7:
            state[plane+48,row,col]=1
        elif val==4:
          row += 1
          state[plane+56,row,col]=1
        elif val==5:
          row += 1
          state[plane+64,row,col]=1
        elif val==6:
          row += 1
          state[plane+72,row,col]=1
        elif val==7:
          row,col = row+1,col+1
          state[plane+56,row,col]=1
        elif val==8:
          row,col = row+1,col+1
          state[plane+64,row,col]=1
        elif val==9:
          row,col = row+1,col+1
          state[plane+72,row,col]=1
        elif val==10:
          row,col = row+1,col-1
          state[plane+56,row,col]=1
        elif val==11:
          row,col = row+1,col-1
          state[plane+64,row,col]=1
        else:
          row,col = row+1,col-1
          state[plane+72,row,col]=1
      elif action<384:
        plane = 24 + (action-104)//28
        if plane>25:
          plane = (plane-25)+71
        pos=list(zip(*np.where(state[plane]==1)))[0]
        if plane==24:
          self.b_right_castle = False
        elif plane==25:
          self.b_left_castle = False
        act = ((action-104)%28)//7
        val = (action-104)%7 + 1
        row,col = pos[0],pos[1]
        if act==0:
          row += val
        elif act==1:
          row -= val
        elif act==2:
          col += val
        else:
          col -= val
      elif action<464:
        plane = 26 + (action-384)//8
        if plane>27:
          plane = (plane-27)+79
        pos=list(zip(*np.where(state[plane]==1)))[0]
        val = (action-384)%8
        row,col = pos[0],pos[1]
        if val==0:
          row+=2
          col+=1
        elif val==1:
          row+=1
          col+=2
        elif val==2:
          row-=1
          col+=2
        elif val==3:
          row-=2
          col+=1
        elif val==4:
          row-=2
          col-=1
        elif val==5:
          row-=1
          col-=2
        elif val==6:
          row+=1
          col-=2
        else:
          row+=2
          col-=1
      elif action<744:
        plane = 28 + (action-464)//28
        if plane>29:
          plane = (plane-29)+87
        pos=list(zip(*np.where(state[plane]==1)))[0]
        act = ((action-464)%28)//7
        val = (action-464)%7 + 1
        row,col = pos[0],pos[1]
        if act==0:
          row,col = row+val,col+val
        elif act==1:
          row,col = row+val,col-val
        elif act==2:
          row,col = row-val,col+val
        else:
          row,col = row-val,col-val
      elif action<1248:
        plane = 30 + (action-744)//56
        if plane>30:
          plane = (plane-30)+63
        pos=list(zip(*np.where(state[plane]==1)))[0]
        act = ((action-744)%56)//7
        val = (action-744)%7 + 1
        row,col = pos[0],pos[1]
        if act==0:
          row += val
        elif act==1:
          row -= val
        elif act==2:
          col += val
        elif act==3:
          col -= val
        elif act==4:
          row,col = row+val,col+val
        elif act==5:
          row,col = row+val,col-val
        elif act==6:
          row,col = row-val,col+val
        else:
          row,col = row-val,col-val
      else:
        plane = 31
        self.b_left_castle = False
        self.b_right_castle = False
        pos=list(zip(*np.where(state[plane]==1)))[0]
        val = (action-1248)
        row,col = pos[0],pos[1]
        if val==0:
          row += 1
        elif val==1:
          row,col = row+1,col+1
        elif val==2:
          col += 1
        elif val==3:
          row,col = row-1,col+1
        elif val==4:
          row -= 1
        elif val==5:
          row,col = row-1,col-1
        elif val==6:
          col -= 1
        elif val==7:
          row,col = row+1,col-1
        elif val==8:
          col += 2
          state[25,0,7]=0
          state[25,0,5]=1
        else:
          col -= 2
          state[24,0,0]=0
          state[24,0,3]=1
      self.w_pawn_en_passant = []
      #if row>7 or col>7:
      #  print(row,col,'row col - action',action,'player',state[96,0,0])
    if state[96,0,0]:
      for er in list(range(16,32))+list(range(64,96)):
        state[er,row,col]=0
    else:
      for er in list(range(16))+list(range(32,64)):
        state[er,row,col]=0
    state[plane,row,col]=1
    #if plane==31 or plane==15:
      #print(plane,'pl')
    state[plane,pos[0],pos[1]]=0
    if state[96,0,0]==1:
      #print(state[96])
      state[96]=np.zeros((1,8,8))
      #print(state[96],'after')
      for i in range(8):
        state[i,0] = np.zeros((1,8))
    else:
      #print(state[96])
      state[96]=np.ones((1,8,8))
      #print(state[96])
      for i in range(8):
        state[i+16,7] = np.zeros((1,8))   
    #print(self.w_pawn_en_passant)
    #print(self.b_pawn_en_passant)
    return state



  def if_check(self,state,player):
    actual = self.actual_state(state)
    if player==1:
      #print('white move')
      pos = list(zip(*np.where(actual==16)))[0]
      #print('king pos',pos)
      
      if pos[0]>1 and (( pos[1]>0 and actual[pos[0]-1][pos[1]-1] in list(range(17,25)) ) or ( pos[1]<7 and actual[pos[0]-1][pos[1]+1] in list(range(17,25)) )):
        return True
      
      for i in range(1,8):
        if pos[0]-i<0:
          break
        if actual[pos[0]-i][pos[1]] not in [0,25,26,31]+list(range(65,81)):
          break
        if actual[pos[0]-i][pos[1]]!=0:
          return True
      
      for i in range(1,8):
        if pos[0]+i>7:
          break
        if actual[pos[0]+i][pos[1]] not in [0,25,26,31]+list(range(65,81)):
          break
        if actual[pos[0]+i][pos[1]]!=0:
          return True

      for i in range(1,8):
        if pos[1]-i<0:
          break
        if actual[pos[0]][pos[1]-i] not in [0,25,26,31]+list(range(65,81)):
          break
        if actual[pos[0]][pos[1]-i]!=0:
          return True
      
      for i in range(1,8):
        if pos[1]+i>7:
          break
        if actual[pos[0]][pos[1]+i] not in [0,25,26,31]+list(range(65,81)):
          break
        if actual[pos[0]][pos[1]+i]!=0:
          return True

      for i in range(1,8):
        if pos[0]-i<0 or pos[1]-i<0:
          break
        if actual[pos[0]-i][pos[1]-i] not in [0,29,30,31]+list(range(65,73))+list(range(89,97)):
          break
        if actual[pos[0]-i][pos[1]-i]!=0:
          return True

      for i in range(1,8):
        if pos[0]-i<0 or pos[1]+i>7:
          break
        if actual[pos[0]-i][pos[1]+i] not in [0,29,30,31]+list(range(65,73))+list(range(89,97)):
          break
        if actual[pos[0]-i][pos[1]+i]!=0:
          return True

      for i in range(1,8):
        if pos[0]+i>7 or pos[1]-i<0:
          break
        if actual[pos[0]+i][pos[1]-i] not in [0,29,30,31]+list(range(65,73))+list(range(89,97)):
          break
        if actual[pos[0]+i][pos[1]-i]!=0:
          return True

      for i in range(1,8):
        if pos[0]+i>7 or pos[1]+i>7:
          break
        if actual[pos[0]+i][pos[1]+i] not in [0,29,30,31]+list(range(65,73))+list(range(89,97)):
          break
        if actual[pos[0]+i][pos[1]+i]!=0:
          return True

      if pos[0]>1 and pos[1]>0 and actual[pos[0]-2][pos[1]-1] in [27,28]+list(range(81,89)):
        return True
      elif pos[0]>0 and pos[1]>1 and actual[pos[0]-1][pos[1]-2] in [27,28]+list(range(81,89)):
        return True
      elif pos[0]<7 and pos[1]>1 and actual[pos[0]+1][pos[1]-2] in [27,28]+list(range(81,89)):
        return True
      elif pos[0]<6 and pos[1]>0 and actual[pos[0]+2][pos[1]-1] in [27,28]+list(range(81,89)):
        return True
      elif pos[0]<6 and pos[1]<7 and actual[pos[0]+2][pos[1]+1] in [27,28]+list(range(81,89)):
        return True
      elif pos[0]<7 and pos[1]<6 and actual[pos[0]+1][pos[1]+2] in [27,28]+list(range(81,89)):
        return True
      elif pos[0]>0 and pos[1]<6 and actual[pos[0]-1][pos[1]+2] in [27,28]+list(range(81,89)):
        return True
      elif pos[0]>1 and pos[1]<7 and actual[pos[0]-2][pos[1]+1] in [27,28]+list(range(81,89)):
        return True

      
      b_king = list(zip(*np.where(actual==32)))[0]
      if b_king[0] in [pos[0],pos[0]-1,pos[0]+1] and b_king[1] in [pos[1],pos[1]-1,pos[1]+1]:
        #print('king true',b_king[0])
        return True
    else:
      #print('black move')
      pos = list(zip(*np.where(actual==32)))[0]
      #print('black king pos',pos)
      
      if pos[0]<6 and (( pos[1]<7 and actual[pos[0]+1][pos[1]+1] in list(range(1,17)) ) or ( pos[1]>0 and actual[pos[0]+1][pos[1]-1] in list(range(1,17)) )):
        return True
      
      for i in range(1,8):
        if pos[0]-i<0:
          break
        if actual[pos[0]-i][pos[1]] not in [0,9,10,15]+list(range(33,49)):
          break
        if actual[pos[0]-i][pos[1]]!=0:
          return True
      
      for i in range(1,8):
        if pos[0]+i>7:
          break
        if actual[pos[0]+i][pos[1]] not in [0,9,10,15]+list(range(33,49)):
          break
        if actual[pos[0]+i][pos[1]]!=0:
          return True

      for i in range(1,8):
        if pos[1]-i<0:
          break
        if actual[pos[0]][pos[1]-i] not in [0,9,10,15]+list(range(33,49)):
          break
        if actual[pos[0]][pos[1]-i]!=0:
          return True
      
      for i in range(1,8):
        if pos[1]+i>7:
          break
        if actual[pos[0]][pos[1]+i] not in [0,9,10,15]+list(range(33,49)):
          break
        if actual[pos[0]][pos[1]+i]!=0:
          return True

      for i in range(1,8):
        if pos[0]-i<0 or pos[1]-i<0:
          break
        if actual[pos[0]-i][pos[1]-i] not in [0,13,14,15]+list(range(33,41))+list(range(57,65)):
          break
        if actual[pos[0]-i][pos[1]-i]!=0:
          return True

      for i in range(1,8):
        if pos[0]-i<0 or pos[1]+i>7:
          break
        if actual[pos[0]-i][pos[1]+i] not in [0,13,14,15]+list(range(33,41))+list(range(57,65)):
          break
        if actual[pos[0]-i][pos[1]+i]!=0:
          return True

      for i in range(1,8):
        if pos[0]+i>7 or pos[1]-i<0:
          break
        if actual[pos[0]+i][pos[1]-i] not in [0,13,14,15]+list(range(33,41))+list(range(57,65)):
          break
        if actual[pos[0]+i][pos[1]-i]!=0:
          return True

      for i in range(1,8):
        if pos[0]+i>7 or pos[1]+i>7:
          break
        if actual[pos[0]+i][pos[1]+i] not in [0,13,14,15]+list(range(33,41))+list(range(57,65)):
          break
        if actual[pos[0]+i][pos[1]+i]!=0:
          return True

      if pos[0]>1 and pos[1]>0 and actual[pos[0]-2][pos[1]-1] in [11,12]+list(range(49,57)):
        return True
      elif pos[0]>0 and pos[1]>1 and actual[pos[0]-1][pos[1]-2] in [11,12]+list(range(49,57)):
        return True
      elif pos[0]<7 and pos[1]>1 and actual[pos[0]+1][pos[1]-2] in [11,12]+list(range(49,57)):
        return True
      elif pos[0]<6 and pos[1]>0 and actual[pos[0]+2][pos[1]-1] in [11,12]+list(range(49,57)):
        return True
      elif pos[0]<6 and pos[1]<7 and actual[pos[0]+2][pos[1]+1] in [11,12]+list(range(49,57)):
        return True
      elif pos[0]<7 and pos[1]<6 and actual[pos[0]+1][pos[1]+2] in [11,12]+list(range(49,57)):
        return True
      elif pos[0]>0 and pos[1]<6 and actual[pos[0]-1][pos[1]+2] in [11,12]+list(range(49,57)):
        return True
      elif pos[0]>1 and pos[1]<7 and actual[pos[0]-2][pos[1]+1] in [11,12]+list(range(49,57)):
        return True

      
      w_king = list(zip(*np.where(actual==16)))[0]
      if w_king[0] in [pos[0],pos[0]-1,pos[0]+1] and w_king[1] in [pos[1],pos[1]-1,pos[1]+1]:
        #print('king true',w_king[0])
        return True
    return False



  def valid_actions(self,state):
    actual = self.actual_state(state)
    
    actions = np.zeros(1258)
    w_rook = [8,9,40,41,42,43,44,45,46,47]
    w_knight = [10,11,48,49,50,51,52,53,54,55]
    w_bishop = [12,13,56,57,58,59,60,61,62,63]
    w_queen = [14,32,33,34,35,36,37,38,39]
    b_rook = [24,25,72,73,74,75,76,77,78,79]
    b_knight = [26,27,80,81,82,83,84,85,86,87]
    b_bishop = [28,29,88,89,90,91,92,93,94,95]
    b_queen = [30,64,65,66,67,68,69,70,71]

    if state[96,0,0]==1:

      for pl in range(1,9):
        pos = list(zip(*np.where(actual==pl)))
        if len(pos)==0:
          continue
        pos = pos[0]
        if pos[0]==6 and actual[pos[0]-2][pos[1]]==0 and actual[pos[0]-1][pos[1]]==0:
          actions[13*(pl-1)+1]=1
        if pos[0]>0 and actual[pos[0]-1][pos[1]]==0:
          actions[13*(pl-1)]=1
          if pos[0]==1:
            actions[13*(pl-1)+4],actions[13*(pl-1)+5],actions[13*(pl-1)+6]=1,1,1
        if pos[0]>0 and pos[1]>0 and actual[pos[0]-1][pos[1]-1] in list(range(17,33))+list(range(65,97)):
          actions[13*(pl-1)+2]=1
          if pos[0]==1:
            actions[13*(pl-1)+7],actions[13*(pl-1)+8],actions[13*(pl-1)+9]=1,1,1
        if pos[0]>0 and pos[1]<7 and actual[pos[0]-1][pos[1]+1] in list(range(17,33))+list(range(65,97)):
          actions[13*(pl-1)+3]=1
          if pos[0]==1:
            actions[13*(pl-1)+10],actions[13*(pl-1)+11],actions[13*(pl-1)+12]=1,1,1
        if pos[0]==3 and pos[1]>0 and actual[pos[0]][pos[1]-1]-1 in self.b_pawn_en_passant:
          actions[13*(pl-1)+2]=1
        if pos[0]==3 and pos[1]<7 and actual[pos[0]][pos[1]+1]-1 in self.b_pawn_en_passant:
          actions[13*(pl-1)+3]=1
      
      for pl in w_rook:
        pos = list(zip(*np.where(actual==(pl+1))))
        if len(pos)==0:
          continue
        pos = pos[0]
        for i in range(1,8):
          if pos[0]-i>-1 and actual[pos[0]-i][pos[1]] not in list(range(1,17))+list(range(33,65)):
            actions[103+28*w_rook.index(pl)+i]
            if actual[pos[0]-i][pos[1]]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[0]+i<8 and actual[pos[0]+i][pos[1]] not in list(range(1,17))+list(range(33,65)):
            actions[103+28*w_rook.index(pl)+i+7]
            if actual[pos[0]+i][pos[1]]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[1]-i>-1 and actual[pos[0]][pos[1]-i] not in list(range(1,17))+list(range(33,65)):
            actions[103+28*w_rook.index(pl)+i+14]
            if actual[pos[0]][pos[1]-i]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[1]+i<8 and actual[pos[0]][pos[1]+i] not in list(range(1,17))+list(range(33,65)):
            actions[103+28*w_rook.index(pl)+i+21]
            if actual[pos[0]][pos[1]+i]!=0:
              break
          else:
            break
          



      for pl in w_knight:
        pos = list(zip(*np.where(actual==pl+1)))
        if len(pos)==0:
          continue
        pos = pos[0]
        if pos[0]-2>-1 and pos[1]-1>-1 and actual[pos[0]-2][pos[1]-1] not in list(range(1,17))+list(range(33,65)):
          actions[383+8*w_knight.index(pl)+1] = 1 
          #print('knight 1',383+8*w_knight.index(pl)+1,'pos',knight[0])
        if pos[0]-1>-1 and pos[1]-2>-1 and actual[pos[0]-1][pos[1]-2] not in list(range(1,17))+list(range(33,65)):
          actions[383+8*w_knight.index(pl)+2] = 1
          #print('knight 2',383+8*w_knight.index(pl)+2,'pos',knight[0])
        if pos[0]+1<8 and pos[1]-2>-1 and actual[pos[0]+1][pos[1]-2] not in list(range(1,17))+list(range(33,65)):
          actions[383+8*w_knight.index(pl)+3] = 1
          #print('knight 3',383+8*w_knight.index(pl)+3,'pos',knight[0])
        if pos[0]+2<8 and pos[1]-1>-1 and actual[pos[0]+2][pos[1]-1] not in list(range(1,17))+list(range(33,65)):
          actions[383+8*w_knight.index(pl)+4] = 1
          #print('knight 4',383+8*w_knight.index(pl)+4,'pos',knight[0])
        if pos[0]+2<8 and pos[1]+1<8 and actual[pos[0]+2][pos[1]+1] not in list(range(1,17))+list(range(33,65)):
          actions[383+8*w_knight.index(pl)+5] = 1
          #print('knight 5',383+8*w_knight.index(pl)+5,'pos',knight[0])
        if pos[0]+1<8 and pos[1]+2<8 and actual[pos[0]+1][pos[1]+2] not in list(range(1,17))+list(range(33,65)):
          actions[383+8*w_knight.index(pl)+6] = 1
          #print('knight 6',383+8*w_knight.index(pl)+6,'pos',knight[0])
        if pos[0]-1>-1 and pos[1]+2<8 and actual[pos[0]-1][pos[1]+2] not in list(range(1,17))+list(range(33,65)):
          actions[383+8*w_knight.index(pl)+7] = 1
          #print('knight 7',383+8*w_knight.index(pl)+7,'pos',knight[0])
        if pos[0]-2>-1 and pos[1]+1<8 and actual[pos[0]-2][pos[1]+1] not in list(range(1,17))+list(range(33,65)):
          actions[383+8*w_knight.index(pl)+8] = 1
          #print('knight 8',383+8*w_knight.index(pl)+8,'pos',knight[0])

      for pl in w_bishop:
        pos = list(zip(*np.where(actual==(pl+1))))
        if len(pos)==0:
          continue
        pos = pos[0]
        for i in range(1,8):
          if pos[0]-i>-1 and pos[1]-i>-1 and actual[pos[0]-i][pos[1]-i] not in list(range(1,17))+list(range(33,65)):
            actions[463+28*w_bishop.index(pl)+i]
            if actual[pos[0]-i][pos[1]-i]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[0]-i>-1 and pos[1]+i<8 and actual[pos[0]-i][pos[1]+i] not in list(range(1,17))+list(range(33,65)):
            actions[463+28*w_bishop.index(pl)+i+7]
            if actual[pos[0]-i][pos[1]+i]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[0]+i<8 and pos[1]-i>-1 and actual[pos[0]+i][pos[1]-i] not in list(range(1,17))+list(range(33,65)):
            actions[463+28*w_bishop.index(pl)+i+14]
            if actual[pos[0]+i][pos[1]-i]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[0]+i<8 and pos[1]+i<8 and actual[pos[0]+i][pos[1]+i] not in list(range(1,17))+list(range(33,65)):
            actions[463+28*w_bishop.index(pl)+i+21]
            if actual[pos[0]+i][pos[1]+i]!=0:
              break
          else:
            break

      for pl in w_queen:
        pos = list(zip(*np.where(actual==(pl+1))))
        if len(pos)==0:
          continue
        pos = pos[0]
        for i in range(1,8):
          if pos[0]-i>-1 and actual[pos[0]-i][pos[1]] not in list(range(1,17))+list(range(33,65)):
            actions[743+56*w_queen.index(pl)+i]
            if actual[pos[0]-i][pos[1]]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[0]+i<8 and actual[pos[0]+i][pos[1]] not in list(range(1,17))+list(range(33,65)):
            actions[743+56*w_queen.index(pl)+i+7]
            if actual[pos[0]+i][pos[1]]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[1]-i>-1 and actual[pos[0]][pos[1]-i] not in list(range(1,17))+list(range(33,65)):
            actions[743+56*w_queen.index(pl)+i+14]
            if actual[pos[0]][pos[1]-i]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[1]+i<8 and actual[pos[0]][pos[1]+i] not in list(range(1,17))+list(range(33,65)):
            actions[743+56*w_queen.index(pl)+i+21]
            if actual[pos[0]][pos[1]+i]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[0]-i>-1 and pos[1]-i>-1 and actual[pos[0]-i][pos[1]-i] not in list(range(1,17))+list(range(33,65)):
            actions[743+56*w_queen.index(pl)+i+28]
            if actual[pos[0]-i][pos[1]-i]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[0]-i>-1 and pos[1]+i<8 and actual[pos[0]-i][pos[1]+i] not in list(range(1,17))+list(range(33,65)):
            actions[743+56*w_queen.index(pl)+i+35]
            if actual[pos[0]-i][pos[1]+i]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[0]+i<8 and pos[1]-i>-1 and actual[pos[0]+i][pos[1]-i] not in list(range(1,17))+list(range(33,65)):
            actions[743+56*w_queen.index(pl)+i+42]
            if actual[pos[0]+i][pos[1]-i]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[0]+i<8 and pos[1]+i<8 and actual[pos[0]+i][pos[1]+i] not in list(range(1,17))+list(range(33,65)):
            actions[743+56*w_queen.index(pl)+i+49]
            if actual[pos[0]+i][pos[1]+i]!=0:
              break
          else:
            break



      pos = list(zip(*np.where(actual==16)))[0]
      #print(pos,'Where is white king?')
      if pos[0]-1>-1 and actual[pos[0]-1][pos[1]] not in list(range(1,17))+list(range(33,65)):
        actions[1247+1]=1
      if pos[0]-1>-1 and pos[1]-1>-1 and actual[pos[0]-1][pos[1]-1] not in list(range(1,17))+list(range(33,65)):
        actions[1247+2]=1
      if pos[1]-1>-1 and actual[pos[0]][pos[1]-1] not in list(range(1,17))+list(range(33,65)):
        actions[1247+3]=1
      if pos[0]+1<8 and pos[1]-1>-1 and actual[pos[0]+1][pos[1]-1] not in list(range(1,17))+list(range(33,65)):
        actions[1247+4]=1
      if pos[0]+1<8 and actual[pos[0]+1][pos[1]] not in list(range(1,17))+list(range(33,65)):
        actions[1247+5]=1
      if pos[0]+1<8 and pos[1]+1<8 and actual[pos[0]+1][pos[1]+1] not in list(range(1,17))+list(range(33,65)):
        actions[1247+6]=1
      if pos[1]+1<8 and actual[pos[0]][pos[1]+1] not in list(range(1,17))+list(range(33,65)):
        actions[1247+7]=1
      if pos[0]-1>-1 and pos[1]+1<8 and actual[pos[0]-1][pos[1]+1] not in list(range(1,17))+list(range(33,65)):
        actions[1247+8]=1
      if self.w_left_castle and actual[7,1]==0 and actual[7,2]==0 and actual[7,3]==0:
        actions[1247+9]=1
      if self.w_right_castle and actual[7,6]==0 and actual[7,5]==0:
        actions[1247+10]=1
        
    else:

      for pl in range(1,9):
        pos = list(zip(*np.where(actual==pl+16)))
        if len(pos)==0:
          continue
        pos = pos[0]
        if pos[0]==1 and actual[pos[0]+2][pos[1]]==0 and actual[pos[0]+1][pos[1]]==0:
          actions[13*(pl-1)+1]=1
        if pos[0]<7 and actual[pos[0]+1][pos[1]]==0:
          actions[13*(pl-1)]=1
          if pos[0]==6:
            actions[13*(pl-1)+4],actions[13*(pl-1)+5],actions[13*(pl-1)+6]=1,1,1
        if pos[0]<7 and pos[1]<7 and actual[pos[0]+1][pos[1]+1] in list(range(1,17))+list(range(33,65)):
          actions[13*(pl-1)+2]=1
          if pos[0]==6:
            actions[13*(pl-1)+7],actions[13*(pl-1)+8],actions[13*(pl-1)+9]=1,1,1
        if pos[0]<7 and pos[1]>0 and actual[pos[0]+1][pos[1]-1] in list(range(1,17))+list(range(33,65)):
          actions[13*(pl-1)+3]=1
          if pos[0]==6:
            actions[13*(pl-1)+10],actions[13*(pl-1)+11],actions[13*(pl-1)+12]=1,1,1
        if pos[0]==4 and pos[1]<7 and actual[pos[0]][pos[1]+1]-1 in self.w_pawn_en_passant:
          actions[13*(pl-1)+2]=1
        if pos[0]==4 and pos[1]>0 and actual[pos[0]][pos[1]-1]-1 in self.w_pawn_en_passant:
          actions[13*(pl-1)+3]=1
      
      for pl in b_rook:
        pos = list(zip(*np.where(actual==(pl+1))))
        if len(pos)==0:
          continue
        pos = pos[0]
        for i in range(1,8):
          if pos[0]+i<8 and actual[pos[0]+i][pos[1]] not in list(range(17,33))+list(range(65,97)):
            actions[103+28*b_rook.index(pl)+i]
            if actual[pos[0]+i][pos[1]]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[0]-i>-1 and actual[pos[0]-i][pos[1]] not in list(range(17,33))+list(range(65,97)):
            actions[103+28*b_rook.index(pl)+i+7]
            if actual[pos[0]-i][pos[1]]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[1]+i<8 and actual[pos[0]][pos[1]+i] not in list(range(1,17))+list(range(33,65)):
            actions[103+28*b_rook.index(pl)+i+14]
            if actual[pos[0]][pos[1]+i]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[1]-i>-1 and actual[pos[0]][pos[1]-i] not in list(range(1,17))+list(range(33,65)):
            actions[103+28*b_rook.index(pl)+i+21]
            if actual[pos[0]][pos[1]-i]!=0:
              break
          else:
            break
          



      for pl in b_knight:
        pos = list(zip(*np.where(actual==pl+1)))
        if len(pos)==0:
          continue
        pos = pos[0]
        if pos[0]+2<8 and pos[1]+1<8 and actual[pos[0]+2][pos[1]+1] not in list(range(17,33))+list(range(65,97)):
          actions[383+8*b_knight.index(pl)+1] = 1 
          #print('knight 1',383+8*w_knight.index(pl)+1,'pos',knight[0])
        if pos[0]+1<8 and pos[1]+2<8 and actual[pos[0]+1][pos[1]+2] not in list(range(17,33))+list(range(65,97)):
          actions[383+8*b_knight.index(pl)+2] = 1
          #print('knight 2',383+8*w_knight.index(pl)+2,'pos',knight[0])
        if pos[0]-1>-1 and pos[1]+2<8 and actual[pos[0]-1][pos[1]+2] not in list(range(17,33))+list(range(65,97)):
          actions[383+8*b_knight.index(pl)+3] = 1
          #print('knight 3',383+8*w_knight.index(pl)+3,'pos',knight[0])
        if pos[0]-2>-1 and pos[1]+1<8 and actual[pos[0]-2][pos[1]+1] not in list(range(17,33))+list(range(65,97)):
          actions[383+8*b_knight.index(pl)+4] = 1
          #print('knight 4',383+8*w_knight.index(pl)+4,'pos',knight[0])
        if pos[0]-2>-1 and pos[1]-1>-1 and actual[pos[0]-2][pos[1]-1] not in list(range(17,33))+list(range(65,97)):
          actions[383+8*b_knight.index(pl)+5] = 1
          #print('knight 5',383+8*w_knight.index(pl)+5,'pos',knight[0])
        if pos[0]-1>-1 and pos[1]-2>-1 and actual[pos[0]-1][pos[1]-2] not in list(range(17,33))+list(range(65,97)):
          actions[383+8*b_knight.index(pl)+6] = 1
          #print('knight 6',383+8*w_knight.index(pl)+6,'pos',knight[0])
        if pos[0]+1<8 and pos[1]-2>-1 and actual[pos[0]+1][pos[1]-2] not in list(range(17,33))+list(range(65,97)):
          actions[383+8*b_knight.index(pl)+7] = 1
          #print('knight 7',383+8*w_knight.index(pl)+7,'pos',knight[0])
        if pos[0]+2<8 and pos[1]-1>-1 and actual[pos[0]+2][pos[1]-1] not in list(range(17,33))+list(range(65,97)):
          actions[383+8*b_knight.index(pl)+8] = 1
          #print('knight 8',383+8*w_knight.index(pl)+8,'pos',knight[0])

      for pl in b_bishop:
        pos = list(zip(*np.where(actual==(pl+1))))
        if len(pos)==0:
          continue
        pos = pos[0]
        for i in range(1,8):
          if pos[0]+i<8 and pos[1]+i<8 and actual[pos[0]+i][pos[1]+i] not in list(range(17,33))+list(range(65,97)):
            actions[463+28*b_bishop.index(pl)+i]
            if actual[pos[0]+i][pos[1]+i]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[0]+i<8 and pos[1]-i>-1 and actual[pos[0]+i][pos[1]-i] not in list(range(17,33))+list(range(65,97)):
            actions[463+28*b_bishop.index(pl)+i+7]
            if actual[pos[0]+i][pos[1]-i]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[0]-i>-1 and pos[1]+i<8 and actual[pos[0]-i][pos[1]+i] not in list(range(17,33))+list(range(65,97)):
            actions[463+28*b_bishop.index(pl)+i+14]
            if actual[pos[0]-i][pos[1]+i]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[0]-i>-1 and pos[1]-i>-1 and actual[pos[0]-i][pos[1]-i] not in list(range(17,33))+list(range(65,97)):
            actions[463+28*b_bishop.index(pl)+i+21]
            if actual[pos[0]-i][pos[1]-i]!=0:
              break
          else:
            break

      for pl in b_queen:
        pos = list(zip(*np.where(actual==(pl+1))))
        if len(pos)==0:
          continue
        pos = pos[0]
        for i in range(1,8):
          if pos[0]+i<8 and actual[pos[0]+i][pos[1]] not in list(range(17,33))+list(range(65,97)):
            actions[743+56*b_queen.index(pl)+i]
            if actual[pos[0]+i][pos[1]]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[0]-i>-1 and actual[pos[0]-i][pos[1]] not in list(range(17,33))+list(range(65,97)):
            actions[743+56*b_queen.index(pl)+i+7]
            if actual[pos[0]-i][pos[1]]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[1]+i<8 and actual[pos[0]][pos[1]+i] not in list(range(17,33))+list(range(65,97)):
            actions[743+56*b_queen.index(pl)+i+14]
            if actual[pos[0]][pos[1]+i]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[1]-i>-1 and actual[pos[0]][pos[1]-i] not in list(range(17,33))+list(range(65,97)):
            actions[743+56*b_queen.index(pl)+i+21]
            if actual[pos[0]][pos[1]-i]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[0]+i<8 and pos[1]+i<8 and actual[pos[0]+i][pos[1]+i] not in list(range(17,33))+list(range(65,97)):
            actions[743+56*b_queen.index(pl)+i+28]
            if actual[pos[0]+i][pos[1]+i]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[0]+i<8 and pos[1]-i>-1 and actual[pos[0]+i][pos[1]-i] not in list(range(17,33))+list(range(65,97)):
            actions[743+56*b_queen.index(pl)+i+35]
            if actual[pos[0]+i][pos[1]-i]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[0]-i>-1 and pos[1]+i<8 and actual[pos[0]-i][pos[1]+i] not in list(range(17,33))+list(range(65,97)):
            actions[743+56*b_queen.index(pl)+i+42]
            if actual[pos[0]-i][pos[1]+i]!=0:
              break
          else:
            break
        for i in range(1,8):
          if pos[0]-i>-1 and pos[1]-i>-1 and actual[pos[0]-i][pos[1]-i] not in list(range(17,33))+list(range(65,97)):
            actions[743+56*b_queen.index(pl)+i+49]
            if actual[pos[0]-i][pos[1]-i]!=0:
              break
          else:
            break



      pos = list(zip(*np.where(actual==32)))[0]
      #print(pos,'Where is b king?')
      if pos[0]+1<8 and actual[pos[0]+1][pos[1]] not in list(range(17,33))+list(range(65,97)):
        actions[1247+1]=1
      if pos[0]+1<8 and pos[1]+1<8 and actual[pos[0]+1][pos[1]+1] not in list(range(17,33))+list(range(65,97)):
        actions[1247+2]=1
      if pos[1]+1<8 and actual[pos[0]][pos[1]+1] not in list(range(17,33))+list(range(65,97)):
        actions[1247+3]=1
      if pos[0]-1>-1 and pos[1]+1<8 and actual[pos[0]-1][pos[1]+1] not in list(range(17,33))+list(range(65,97)):
        actions[1247+4]=1
      if pos[0]-1>-1 and actual[pos[0]-1][pos[1]] not in list(range(17,33))+list(range(65,97)):
        actions[1247+5]=1
      if pos[0]-1>-1 and pos[1]-1>-1 and actual[pos[0]-1][pos[1]-1] not in list(range(17,33))+list(range(65,97)):
        actions[1247+6]=1
      if pos[1]-1>-1 and actual[pos[0]][pos[1]-1] not in list(range(17,33))+list(range(65,97)):
        actions[1247+7]=1
      if pos[0]+1<8 and pos[1]-1>-1 and actual[pos[0]+1][pos[1]-1] not in list(range(17,33))+list(range(65,97)):
        actions[1247+8]=1
      if self.b_left_castle and actual[0,5]==0 and actual[0,6]==0:
        actions[1247+9]=1
      if self.b_right_castle and actual[0,1]==0 and actual[0,2]==0 and actual[0,3]==0:
        actions[1247+10]=1

      
    
    #print(state[15],'before loop chk',)
    #print(state[31],'before loop chk b')
    #print(actions[453],'453 before loop - player',state[96,0,0])
    va = list(zip(*np.where(actions==1)))
    for i in va:
      st = copy.deepcopy(state)
      if self.if_check(self.apply_action(st,i[0]),state[96,0,0]):
        actions[i[0]]=0
      else:
        continue
    return actions

  def actual_state(self,state):
    actual = np.zeros((8,8))
    #count_wp = 8
    #count_bp = 8
    w_rook = [8,9,40,41,42,43,44,45,46,47]
    w_knight = [10,11,48,49,50,51,52,53,54,55]
    w_bishop = [12,13,56,57,58,59,60,61,62,63]
    w_queen = [14,32,33,34,35,36,37,38,39]
    b_rook = [24,25,72,73,74,75,76,77,78,79]
    b_knight = [26,27,80,81,82,83,84,85,86,87]
    b_bishop = [28,29,88,89,90,91,92,93,94,95]
    b_queen = [30,64,65,66,67,68,69,70,71]
    for i in range(8):
      w_pawn = list(zip(*np.where(state[i]==1)))
      if len(w_pawn)!=0:
        #count_wp-=1
        actual[w_pawn[0][0],w_pawn[0][1]]=i+1
      b_pawn = list(zip(*np.where(state[i+16]==1)))
      if len(b_pawn)!=0:
        #count_bp-=1
        actual[b_pawn[0][0],b_pawn[0][1]]=i+16+1

    for i in range(len(w_rook)):
      rook = list(zip(*np.where(state[w_rook[i]]==1)))
      if len(rook)!=0:
        actual[rook[0][0],rook[0][1]]=w_rook[i]+1
    for i in range(len(b_rook)):
      rook = list(zip(*np.where(state[b_rook[i]]==1)))
      if len(rook)!=0:
        actual[rook[0][0],rook[0][1]]=b_rook[i]+1

    for i in range(len(w_knight)):
      knight = list(zip(*np.where(state[w_knight[i]]==1)))
      if len(knight)!=0:
        actual[knight[0][0],knight[0][1]]=w_knight[i]+1
    for i in range(len(b_knight)):
      knight = list(zip(*np.where(state[b_knight[i]]==1)))
      if len(knight)!=0:
        actual[knight[0][0],knight[0][1]]=b_knight[i]+1 

    for i in range(len(w_bishop)):
      bishop = list(zip(*np.where(state[w_bishop[i]]==1)))
      if len(bishop)!=0:
        actual[bishop[0][0],bishop[0][1]]=w_bishop[i]+1
    for i in range(len(b_bishop)):
      bishop = list(zip(*np.where(state[b_bishop[i]]==1)))
      if len(bishop)!=0:
        actual[bishop[0][0],bishop[0][1]]=b_bishop[i]+1

    for i in range(len(w_queen)):
      queen = list(zip(*np.where(state[w_queen[i]]==1)))
      if len(queen)!=0:
        actual[queen[0][0],queen[0][1]]=w_queen[i]+1
    for i in range(len(b_queen)):
      queen = list(zip(*np.where(state[b_queen[i]]==1)))
      if len(queen)!=0:
        actual[queen[0][0],queen[0][1]]=b_queen[i]+1

    w_king = list(zip(*np.where(state[15]==1)))[0]
    actual[w_king[0],w_king[1]]=16
    b_king = list(zip(*np.where(state[31]==1)))[0]
    actual[b_king[0],b_king[1]]=32
    return actual

  def is_terminal(self,state,valid,in_states,episode_states):
    
    rep_count = dict()
    for e in in_states+episode_states+[state]:
      temp = e.tobytes()
      if temp in rep_count:
        rep_count[temp]+=1
      else:
        rep_count[temp]=1
    if (np.array(list(rep_count.values()))>=5).any():
      print('rep 5 times')
      return True,0
    if len(in_states+episode_states)>=200:
      print('more than 200')
      return True,0
    if (valid==1).any():
      return False,None
    elif self.if_check(state,state[96,0,0]):
      print('chkmate')
      return True,1
    else:
      print('draw by stale')
      return True,0
    
    






In [ ]:
r = Representation()
st = copy.deepcopy(r.start_state)
temp = r.apply_action(st,1)
temp = r.apply_action(temp,1)
print(r.w_pawn_en_passant)
print(r.b_pawn_en_passant)

[]
[16]


In [ ]:

class PolicyNetwork(tf.keras.Model,Representation):

  def __init__(self, name=None):
    tf.keras.Model.__init__(self)
    Representation.__init__(self)
    
    self.conv = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn = tf.keras.layers.BatchNormalization()
    
    self.conv1_1 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn1_1 = tf.keras.layers.BatchNormalization()
    self.conv1_2 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn1_2 = tf.keras.layers.BatchNormalization()

    self.conv2_1 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn2_1 = tf.keras.layers.BatchNormalization()
    self.conv2_2 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn2_2 = tf.keras.layers.BatchNormalization()
    
    self.conv3_1 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn3_1 = tf.keras.layers.BatchNormalization()
    self.conv3_2 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn3_2 = tf.keras.layers.BatchNormalization()
    
    self.conv4_1 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn4_1 = tf.keras.layers.BatchNormalization()
    self.conv4_2 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn4_2 = tf.keras.layers.BatchNormalization()
    
    self.conv5_1 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn5_1 = tf.keras.layers.BatchNormalization()
    self.conv5_2 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn5_2 = tf.keras.layers.BatchNormalization()
    
    self.conv6_1 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn6_1 = tf.keras.layers.BatchNormalization()
    self.conv6_2 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn6_2 = tf.keras.layers.BatchNormalization()
    
    self.conv7_1 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn7_1 = tf.keras.layers.BatchNormalization()
    self.conv7_2 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn7_2 = tf.keras.layers.BatchNormalization()
    
    self.conv8_1 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn8_1 = tf.keras.layers.BatchNormalization()
    self.conv8_2 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn8_2 = tf.keras.layers.BatchNormalization()
    
    self.conv9_1 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn9_1 = tf.keras.layers.BatchNormalization()
    self.conv9_2 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn9_2 = tf.keras.layers.BatchNormalization()
    
    self.conv10_1 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn10_1 = tf.keras.layers.BatchNormalization()
    self.conv10_2 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn10_2 = tf.keras.layers.BatchNormalization()
    
    self.conv11_1 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn11_1 = tf.keras.layers.BatchNormalization()
    self.conv11_2 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn11_2 = tf.keras.layers.BatchNormalization()
    
    self.conv12_1 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn12_1 = tf.keras.layers.BatchNormalization()
    self.conv12_2 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn12_2 = tf.keras.layers.BatchNormalization()
    
    self.conv13_1 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn13_1 = tf.keras.layers.BatchNormalization()
    self.conv13_2 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn13_2 = tf.keras.layers.BatchNormalization()
    
    self.conv14_1 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn14_1 = tf.keras.layers.BatchNormalization()
    self.conv14_2 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn14_2 = tf.keras.layers.BatchNormalization()
    
    self.conv15_1 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn15_1 = tf.keras.layers.BatchNormalization()
    self.conv15_2 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn15_2 = tf.keras.layers.BatchNormalization()
    
    self.conv16_1 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn16_1 = tf.keras.layers.BatchNormalization()
    self.conv16_2 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn16_2 = tf.keras.layers.BatchNormalization()
    
    self.conv17_1 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn17_1 = tf.keras.layers.BatchNormalization()
    self.conv17_2 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn17_2 = tf.keras.layers.BatchNormalization()
    
    self.conv18_1 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn18_1 = tf.keras.layers.BatchNormalization()
    self.conv18_2 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn18_2 = tf.keras.layers.BatchNormalization()
    
    self.conv19_1 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn19_1 = tf.keras.layers.BatchNormalization()
    self.conv19_2 = tf.keras.layers.Conv2D(256,3,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn19_2 = tf.keras.layers.BatchNormalization()
    
    self.conv_h1 = tf.keras.layers.Conv2D(2,1,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn_h1 = tf.keras.layers.BatchNormalization()
    self.dense_h1 = tf.keras.layers.Dense(1258,kernel_regularizer=tf.keras.regularizers.L2(0.0001))

    self.conv_h2 = tf.keras.layers.Conv2D(1,1,padding='same',kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    self.bn_h2 = tf.keras.layers.BatchNormalization()
    self.dense_h2 = tf.keras.layers.Dense(1,kernel_regularizer=tf.keras.regularizers.L2(0.0001))
    

  def call(self,input_tensor,training=False):
    
    input = self.conv(input_tensor)
    input = self.bn(input,training=training)
    input = tf.nn.relu(input)
    
    x = self.conv1_1(input)
    x = self.bn1_1(x,training=training)
    x = self.conv1_2(x)
    x = self.bn1_2(x,training=training)
    x = tf.keras.layers.add([x,input])
    block_1 = tf.nn.relu(x)

    x = self.conv2_1(block_1)
    x = self.bn2_1(x,training=training)
    x = self.conv2_2(x)
    x = self.bn2_2(x,training=training)
    x = tf.keras.layers.add([x,block_1])
    block_2 = tf.nn.relu(x)

    x = self.conv3_1(block_2)
    x = self.bn3_1(x,training=training)
    x = self.conv3_2(x)
    x = self.bn3_2(x,training=training)
    x = tf.keras.layers.add([x,block_2])
    block_3 = tf.nn.relu(x)
    
    x = self.conv4_1(block_3)
    x = self.bn4_1(x,training=training)
    x = self.conv4_2(x)
    x = self.bn4_2(x,training=training)
    x = tf.keras.layers.add([x,block_3])
    block_4 = tf.nn.relu(x)
    
    x = self.conv5_1(block_4)
    x = self.bn5_1(x,training=training)
    x = self.conv5_2(x)
    x = self.bn5_2(x,training=training)
    x = tf.keras.layers.add([x,block_4])
    block_5 = tf.nn.relu(x)
    
    x = self.conv6_1(block_5)
    x = self.bn6_1(x,training=training)
    x = self.conv6_2(x)
    x = self.bn6_2(x,training=training)
    x = tf.keras.layers.add([x,block_5])
    block_6 = tf.nn.relu(x)
    
    x = self.conv7_1(block_6)
    x = self.bn7_1(x,training=training)
    x = self.conv7_2(x)
    x = self.bn7_2(x,training=training)
    x = tf.keras.layers.add([x,block_6])
    block_7 = tf.nn.relu(x)

    x = self.conv8_1(block_7)
    x = self.bn8_1(x,training=training)
    x = self.conv8_2(x)
    x = self.bn8_2(x,training=training)
    x = tf.keras.layers.add([x,block_7])
    block_8 = tf.nn.relu(x)

    x = self.conv9_1(block_8)
    x = self.bn9_1(x,training=training)
    x = self.conv9_2(x)
    x = self.bn9_2(x,training=training)
    x = tf.keras.layers.add([x,block_8])
    block_9 = tf.nn.relu(x)

    x = self.conv10_1(block_9)
    x = self.bn10_1(x,training=training)
    x = self.conv10_2(x)
    x = self.bn10_2(x,training=training)
    x = tf.keras.layers.add([x,block_9])
    block_10 = tf.nn.relu(x)

    x = self.conv11_1(block_10)
    x = self.bn11_1(x,training=training)
    x = self.conv11_2(x)
    x = self.bn11_2(x,training=training)
    x = tf.keras.layers.add([x,block_10])
    block_11 = tf.nn.relu(x)

    x = self.conv12_1(block_11)
    x = self.bn12_1(x,training=training)
    x = self.conv12_2(x)
    x = self.bn12_2(x,training=training)
    x = tf.keras.layers.add([x,block_11])
    block_12 = tf.nn.relu(x)

    x = self.conv13_1(block_12)
    x = self.bn13_1(x,training=training)
    x = self.conv13_2(x)
    x = self.bn13_2(x,training=training)
    x = tf.keras.layers.add([x,block_12])
    block_13 = tf.nn.relu(x)
    
    x = self.conv14_1(block_13)
    x = self.bn14_1(x,training=training)
    x = self.conv14_2(x)
    x = self.bn14_2(x,training=training)
    x = tf.keras.layers.add([x,block_13])
    block_14 = tf.nn.relu(x)

    x = self.conv15_1(block_14)
    x = self.bn15_1(x,training=training)
    x = self.conv15_2(x)
    x = self.bn15_2(x,training=training)
    x = tf.keras.layers.add([x,block_14])
    block_15 = tf.nn.relu(x)

    x = self.conv16_1(block_15)
    x = self.bn16_1(x,training=training)
    x = self.conv16_2(x)
    x = self.bn16_2(x,training=training)
    x = tf.keras.layers.add([x,block_15])
    block_16 = tf.nn.relu(x)

    x = self.conv17_1(block_16)
    x = self.bn17_1(x,training=training)
    x = self.conv17_2(x)
    x = self.bn17_2(x,training=training)
    x = tf.keras.layers.add([x,block_16])
    block_17 = tf.nn.relu(x)

    x = self.conv18_1(block_17)
    x = self.bn18_1(x,training=training)
    x = self.conv18_2(x)
    x = self.bn18_2(x,training=training)
    x = tf.keras.layers.add([x,block_17])
    block_18 = tf.nn.relu(x)

    x = self.conv19_1(block_18)
    x = self.bn19_1(x,training=training)
    x = self.conv19_2(x)
    x = self.bn19_2(x,training=training)
    x = tf.keras.layers.add([x,block_18])
    block_19 = tf.nn.relu(x)
    
    h1 = self.conv_h1(block_19)
    h1 = self.bn_h1(h1,training=training)
    h1 = tf.keras.layers.Flatten()(h1)
    h1 = self.dense_h1(h1)
    h1 = tf.exp(h1)
    valid = []
    for b in input_tensor.numpy().reshape((-1,97,8,8)):
      valid.append(self.valid_actions(b))
    valid = np.array(valid)
    h1 = tf.math.multiply(valid.astype('float32'),h1)
    policy = h1/tf.reduce_sum(h1)

    h2 = self.conv_h2(block_19)
    h2 = self.bn_h2(h2,training=training)
    h2 = tf.keras.layers.Flatten()(h2)
    h2 = self.dense_h2(h2)
    value = tf.nn.tanh(h2)

    return policy,value




In [ ]:
model = PolicyNetwork()
model(model.start_state.reshape((1,8,8,97)))


In [ ]:
class treenode:
  def __init__(self,state,value):
    self.state=state
    self.value=value
    self.child=[]
    self.parent=None
  def add_child(self,child):
    self.child.append(child)
    child.parent=self

class tree:
  def __init__(self,state,value):
    self.head=treenode(state,value)
  def insert(self,state,value):
    for i in self.head.child:
      if (i.state==state).all():
        self.head = i
        #print('Mcts or step?')
        return
    #print('Mcts or step? not inn..')
    child=treenode(state,value)
    self.head.add_child(child)
    self.head = child
  def parent(self):
    parent = self.head.parent
    self.head=self.head.parent
    return parent




In [ ]:
def loss(model, s, p, z, training = True, reg_parameter = 10**-4):
  #reg_loss added to model_losses when defined vars.
  p_model,v = model(s, training = training)
  #print('In Loss fn shapes.....pm v',p_model.shape, v.shape)
  p_model = ma.log(p_model.numpy()).astype('float64')
  p_model.mask = ma.nomask
  #print('In Loss fn shapes.....pm v',p_model.shape)
  loss1 = tf.reduce_sum((z-v)**2)
  loss2 =  tf.reduce_sum(- tf.cast(p*p_model,tf.float32))
  reg_loss = 0.5*tf.reduce_sum(model.losses)
  return loss1+reg_loss, loss2+reg_loss


def grad(model, s, p, z, training = True):
  with tf.GradientTape() as tape:
    loss1, loss2 = loss(model, s, p, z, training)
  loss_val = loss1+loss2
  return loss_val, tape.gradient([loss1, loss2], model.trainable_variables)

#optimizer = tf.keras.optimizers.SGD(learning_rate=0.01,momentum=0.9)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

#optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)


In [ ]:

class Simulations(Representation):

  def __init__(self, network = '/content/drive/My Drive/RL/POLICY_SELF_PLAY/chkpt', train_network = '/content/drive/My Drive/RL/POLICY_UNDER_TRAINING/chkpt'):
    Representation.__init__(self)
    self.simulations = 1
    self.leaf_depth = 3
    self.c = 1
    self.no_of_init_games = 3
    
    self.episode_states = [] 
    self.episode_policies = []
    self.episode_rewards = []
    self.episode_actions = []
    self.last_state = None
    self.last_action = None
    self.P = None
    self.Q = None
    self.N = None
    self.W = None
    self.V = None

    self.network = PolicyNetwork()
    self.training_network = PolicyNetwork()
    self.network.load_weights(network)
    self.training_network.load_weights(train_network)
    #self.learning_rate = 0.01
    #self.optimizer = tf.keras.optimizers.SGD(learning_rate=self.learning_rate,momentum=0.9)


  def MCTS(self, state,P,Q,N,W,V,network, sim = None, eval_network = None):

    if sim==None:
      sim = self.simulations

    if eval_network==None:
      eval_network = network

    if len(self.episode_states)<10:
      temp = 1.0
    else:
      temp = 0.125
    terminated = False
    dirichlet_noise = np.random.dirichlet(0.03*np.ones((1258)))
    epsilon = 0.25
    P.head.value = (1-epsilon)*P.head.value+epsilon*dirichlet_noise
    

    for s in range(sim):
      current_state = copy.deepcopy(state)
      in_actions,in_states=[],[]
      for l in range(self.leaf_depth):
        #print(' in leaf dpt l->',l)
        found=False
        va = self.valid_actions(current_state)

        U = self.c*(P.head.value*np.sqrt(np.sum(N.head.value)))/(1+N.head.value)
        
        current_action = np.argmax(ma.masked_equal((Q.head.value+U)*va,value=0))
        #print('In MCTS current a',current_action,'q val',Q.head.value[current_action],'va',va[current_action])
        #print(sorted((Q.head.value)*va,reverse=True)[:5])
        
        in_actions.append(current_action)
        in_states.append(current_state)
        N.head.value[current_action]+=1
        new_state = self.apply_action(copy.deepcopy(current_state),current_action)
        if self.is_terminal(new_state,va,in_states,self.episode_states)[0]:
          terminated = True
          break
        else:
          current_state = copy.deepcopy(new_state)

        for i in range(len(P.head.child)):
          if (P.head.child[i].state == current_state).all():
            P.head = P.head.child[i]
            Q.head = Q.head.child[i]
            N.head = N.head.child[i]
            W.head = W.head.child[i]
            V.head = V.head.child[i]
            found=True
            break
        if found:
          continue
        if s%2!=0:
          current_policy,current_value = eval_network(current_state.reshape((1,8,8,97)))
        else:
          current_policy,current_value = network(current_state.reshape((1,8,8,97)))
        current_policy,current_value = current_policy.numpy(),current_value.numpy()
        P.insert(copy.deepcopy(current_state), current_policy.reshape((1258)))
        #print('insert after P bef N')
        N.insert(copy.deepcopy(current_state), np.zeros((1258)))
        #print('insert before Q')
        Q.insert(copy.deepcopy(current_state), (current_policy*current_value).reshape((1258)))
        #print('insert bef W')
        W.insert(copy.deepcopy(current_state), np.zeros((1258)))
        #print('insert bef V')
        V.insert(copy.deepcopy(current_state), current_value.reshape((1)))
        #print('insert after V bef P')

      
      if not terminated:
        P.parent()
        Q.parent()
        N.parent()
        W.parent()
      for st in range(1,l+2):
        #print(W.head.value[in_actions[-st]],'W is ? or which is none ')
        #print(V.head.value[0],'V is none...')
        W.head.value[in_actions[-st]] += V.head.value[0]
        Q.head.value[in_actions[-st]] = W.head.value[in_actions[-st]]/N.head.value[in_actions[-st]]
        if st == l+1:
          break

        P.parent()
        Q.parent()
        N.parent()
        W.parent()
      if not terminated:
        V.parent()
      for st in range(1,l+1):
        V.parent()
        #print(Q.head.value[in_actions[-st]],'Qval parent after update')
      #print(N.head.value,'n val after sim ,sum->',np.sum(N.head.value))
      if terminated:
        break
    search_policy = N.head.value**(1/temp)/np.sum(N.head.value**(1/temp))
    return search_policy

  def agent_start(self):
    
    self.last_state = self.start_state
    #print(list(zip(*np.where(self.valid_actions(self.last_state)==1))))
    policy,value = self.network(self.last_state.reshape((1,8,8,97)))
    policy,value = policy.numpy(),value.numpy()
    
    self.P = tree(copy.deepcopy(self.last_state), policy.reshape((1258)))
    self.N = tree(copy.deepcopy(self.last_state), np.zeros((1258)))
    self.Q = tree(copy.deepcopy(self.last_state), (policy*value).reshape((1258)))
    self.W = tree(copy.deepcopy(self.last_state), np.zeros((1258)))
    self.V = tree(copy.deepcopy(self.last_state), value.reshape((1)))

  def agent_step(self):

    while True:
      search_policy = self.MCTS(self.last_state,self.P,self.Q,self.N,self.W,self.V,self.network)
      self.last_action = np.argmax(search_policy)
      self.episode_states.append(self.last_state)
      self.episode_policies.append(search_policy)
      self.episode_actions.append(self.last_action)

      #print('search pol rev sort last 5',sorted(search_policy,reverse=True)[:5])
      #print('search pol for action->',search_policy[self.last_action])
      #print('in step action->',self.last_action)
      self.last_state = self.apply_action(self.last_state,self.last_action)

      valid = self.valid_actions(self.last_state)
      terminal,reward = self.is_terminal(self.last_state,valid,[],self.episode_states)
      if terminal:
        a = np.zeros((len(self.episode_states)))
        a[::2] = reward
        a[1::2] = -1*reward
        a = a[::-1]
        self.episode_rewards = a
        break
      else:
        #print('step?? insert nxt')
        self.P.insert(self.last_state,None)
        self.P.head.parent = None
        self.Q.insert(self.last_state,None)
        self.Q.head.parent = None
        self.N.insert(self.last_state,None)
        self.N.head.parent = None
        self.W.insert(self.last_state,None)
        self.W.head.parent = None
        self.V.insert(self.last_state,None)
        self.V.head.parent = None
        #self.last_state = self.start_state

  def self_play_and_weights_update(self, start_epoch, end_epoch, batch_size = 1024, mini_batch = 128):
    for i in range(start_epoch, end_epoch):
      '''self.agent_start()
      self.agent_step()
      for e in range(len(self.episode_states)):
        data_array = np.array([self.episode_states[e],self.episode_policies[e],self.episode_rewards[e]])
        np.save('/content/drive/My Drive/RL/Data/data_array'+'-R-'+ str(i)+'-'+ str(e), data_array)
      print('selfplay:',i)
      no_of_old_f = max(len(os.listdir('/content/drive/My Drive/RL/Data')) - 25000, 0)
      if no_of_old_f:
        break
        time_sorted = sorted([os.path.abspath('/content/drive/My Drive/RL/Data'+'/'+f) for f in os.listdir('/content/drive/My Drive/RL/Data')], key=os.path.getctime)
        for rm_old in range(no_of_old_f):
          oldest_file = time_sorted[rm_old]
          os.remove(oldest_file)'''
      batch_data = np.random.choice(os.listdir('/content/ex_games'), size = batch_size)
      s,p,z=[],[],[]
      for data in batch_data:
        temp = np.load('/content/ex_games'+'/'+data, allow_pickle = True)
        s.append(temp[0].reshape((8,8,97)))
        p.append(temp[1].reshape((1258)))
        z.append(temp[2].reshape((1)))
      s,p,z = np.array(s),np.array(p),np.array(z)
      #print('Data collected: shapes.......',p.shape, z.shape)
      for mini in range(int(batch_size/mini_batch)):  
        #print('Data collected: shapes.......',p[mini*mini_batch : mini*mini_batch+mini_batch].shape, z[mini*mini_batch : mini*mini_batch+mini_batch].shape)
        loss_value, grads = grad(self.training_network, s[mini*mini_batch : mini*mini_batch+mini_batch],p[mini*mini_batch : mini*mini_batch+mini_batch],z[mini*mini_batch : mini*mini_batch+mini_batch])
        print("Step: {}, Loss: {}".format(optimizer.iterations.numpy(), loss_value.numpy()/mini_batch))
        optimizer.apply_gradients(zip(grads, self.training_network.trainable_variables))
        #print("Step: {}, Loss: {}".format(optimizer.iterations.numpy(), loss(self.training_network, s[mini*mini_batch : mini*mini_batch+mini_batch],p[mini*mini_batch : mini*mini_batch+mini_batch],z[mini*mini_batch : mini*mini_batch+mini_batch],  training=True)))
      

  def evaluator(self, no_of_games = 10, mcts_sim = 1):
    #self.network, self.training_network
    #evengame = self.network, oddgame = self.training_network
    for games in range(no_of_games):
      eval_episode_states = []
      eval_net = self.network
      eval_train_net = self.training_network
      eval_start = self.start_state
      eval_last_state = eval_start
      if games%2==0:
        policy,value = eval_net(eval_last_state.reshape((1,8,8,97)))
        policy,value = policy.numpy(),value.numpy()
        P = tree(copy.deepcopy(eval_last_state), policy.reshape((1258)))
        N = tree(copy.deepcopy(eval_last_state), np.zeros((1258)))
        Q = tree(copy.deepcopy(eval_last_state), (policy*value).reshape((1258)))
        W = tree(copy.deepcopy(eval_last_state), np.zeros((1258)))
        V = tree(copy.deepcopy(eval_last_state), value.reshape((1)))
        iter = 0
        while True:
          iter+=1
          if iter%2!=0:
            net = eval_net
            enet = eval_train_net
          else:
            net = eval_train_net
            enet = eval_net
          eval_episode_states.append(eval_last_state)
          search_policy = self.MCTS(eval_last_state,P,Q,N,W,V,net,mcts_sim, enet)
          eval_last_action = np.argmax(search_policy)
          eval_last_state = self.apply_action(eval_last_state, eval_last_action)
          valid = self.valid_actions(eval_last_state)
          terminal,reward = self.is_terminal(eval_last_state,valid,[],eval_episode_states)
          if terminal:
            if len(eval_episode_states)%2==0:
              reward*=-1
            print('Terminated:',games%2,'Reward:',reward)
            break
          else:
            P.insert(eval_last_state,None)
            P.head.parent = None
            Q.insert(eval_last_state,None)
            Q.head.parent = None
            N.insert(eval_last_state,None)
            N.head.parent = None
            W.insert(eval_last_state,None)
            W.head.parent = None
            V.insert(eval_last_state,None)
            V.head.parent = None
      else:
        policy,value = eval_train_net(eval_last_state.reshape((1,8,8,97)))
        policy,value = policy.numpy(),value.numpy()
        P = tree(copy.deepcopy(eval_last_state), policy.reshape((1258)))
        N = tree(copy.deepcopy(eval_last_state), np.zeros((1258)))
        Q = tree(copy.deepcopy(eval_last_state), (policy*value).reshape((1258)))
        W = tree(copy.deepcopy(eval_last_state), np.zeros((1258)))
        V = tree(copy.deepcopy(eval_last_state), value.reshape((1)))
        iter = 0
        while True:
          iter+=1
          if iter%2==0:
            net = eval_net
            enet = eval_train_net
          else:
            net = eval_train_net
            enet = eval_net
          eval_episode_states.append(eval_last_state)
          search_policy = self.MCTS(eval_last_state,P,Q,N,W,V,net,mcts_sim)
          eval_last_action = np.argmax(search_policy)
          eval_last_state = self.apply_action(eval_last_state, eval_last_action)
          valid = self.valid_actions(eval_last_state)
          terminal,reward = self.is_terminal(eval_last_state,valid,[],eval_episode_states)
          if terminal:
            if len(eval_episode_states)%2==0:
              reward*=-1
            print('Terminated:',games%2,'Reward:',reward)
            break
          else:
            P.insert(eval_last_state,None)
            P.head.parent = None
            Q.insert(eval_last_state,None)
            Q.head.parent = None
            N.insert(eval_last_state,None)
            N.head.parent = None
            W.insert(eval_last_state,None)
            W.head.parent = None
            V.insert(eval_last_state,None)
            V.head.parent = None
    

      

In [ ]:
sim = Simulations()
sim.self_play_and_weights_update(0, 200, 2048, 256)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Step: 1600, Loss: 5.23390007019043
Step: 1601, Loss: 5.0887980461120605
Step: 1602, Loss: 4.811291694641113
Step: 1603, Loss: 4.979675769805908
Step: 1604, Loss: 5.023958683013916
Step: 1605, Loss: 5.075577735900879
Step: 1606, Loss: 4.822131633758545
Step: 1607, Loss: 5.10040807723999
Step: 1608, Loss: 5.350536346435547
Step: 1609, Loss: 5.074010372161865
Step: 1610, Loss: 5.075831413269043
Step: 1611, Loss: 5.250559329986572
Step: 1612, Loss: 5.506289482116699
Step: 1613, Loss: 5.216933250427246
Step:

In [ ]:
sim.training_network.save_weights('/content/drive/My Drive/RL/POLICY_UNDER_TRAINING/chkpt')

In [ ]:
batch_data = np.random.choice(os.listdir('/content/drive/My Drive/RL/ex_games'), size = 1)

In [ ]:
!unzip /content/ex_games-20201018T144641Z-001.zip

In [ ]:
s = Simulations()
s.evaluator()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Terminated: 0 Re

In [ ]:
"""%load_ext tensorboard
%tensorboard --logdir {logs_base_dir}
"""

In [ ]:
import time
s = Simulations()
c = time.time()
s.agent_start()
print('step starts')
s.agent_step()
print(time.time()-c)



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

step starts
2167.0994606018066


In [ ]:
#print(s.actual_state(np.frombuffer(s.episode_states[-1]).reshape(97,8,8)))

[[25. 27. 29. 31. 32. 30.  0. 26.]
 [ 0.  0. 19.  0. 21. 22.  0.  0.]
 [ 0.  0.  0.  0.  0. 28.  0.  0.]
 [17.  1.  0. 20.  0.  0. 23. 24.]
 [ 0.  0.  0.  4.  0.  0.  7.  8.]
 [ 0.  0.  0.  0.  0.  0.  0. 12.]
 [ 0.  2.  3.  0.  5.  6.  0.  0.]
 [ 9. 11. 13. 15. 16. 14.  0. 10.]]


In [ ]:
print(s.actual_state(np.frombuffer(s.episode_states[-1]).reshape(97,8,8)))

[[25.  0.  0. 31.  0. 30.  0. 26.]
 [ 0. 18.  0.  0. 21.  0. 23. 24.]
 [ 0.  0.  0.  0. 32.  0.  0. 28.]
 [17.  0. 19.  0.  0.  0.  0. 16.]
 [ 1.  0.  3.  0.  0.  6.  0.  0.]
 [ 0.  0.  2.  0.  5.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 9.  0. 13. 15.  0. 14.  0. 10.]]


In [ ]:
print(s.episode_actions[44])

1251


In [ ]:
print(np.frombuffer(s.episode_policies[44])[1251])

1.0


In [ ]:
len(s.episode_states)

55

In [ ]:
print(s.actual_state(np.frombuffer(s.last_state).reshape(97,8,8)))

[[25.  0. 29. 31.  0.  0.  0. 26.]
 [17.  0. 19.  0.  0. 22.  0.  0.]
 [28.  0.  3.  0. 32. 23.  0.  0.]
 [ 0. 18.  0.  0. 21.  0.  0.  0.]
 [ 0.  2.  0. 20.  5.  0.  0.  0.]
 [ 0.  0. 27.  4.  0.  0.  0. 12.]
 [ 0.  0.  0.  0. 16.  6.  7.  8.]
 [ 9.  0.  0. 15.  0. 14.  0. 10.]]


In [ ]:
s.valid_actions(np.frombuffer(s.episode_states[44]).reshape(97,8,8))[1251]

(5, 5) Where is white king?


0.0

In [ ]:
print(s.episode_rewards[0])

0.0


In [ ]:
##Match between self play and trained nn: 
r = Representation()
sp = PolicyNetwork()
tr = PolicyNetwork()
sp.load_weights('/content/drive/My Drive/RL/POLICY_UNDER_TRAINING/chkpt')
tr.load_weights('/content/drive/My Drive/RL/POLICY_UNDER_TRAINING/chkpt')

In [ ]:
#VS cont....state,valid,in_states,episode_states
state = r.start_state
ep = [copy.deepcopy(state)]

for i in range(200):
  policy,val = sp(state.reshape((1,8,8,97)))
  act = np.argmax(policy.numpy())
  print(act,'wmove')
  state = r.apply_action(state,act)
  print(r.actual_state(state))
  terminal,reward = r.is_terminal(state, r.valid_actions(state), [], ep)
  if terminal:
    print(reward)
    break
  ep.append(copy.deepcopy(state))
  policy2,val2 = tr(state.reshape((1,8,8,97)))
  act2 = np.argmax(policy2.numpy())
  print(act2,'bmove')
  state = r.apply_action(state,act2) 
  print(r.actual_state(state))
  terminal,reward = r.is_terminal(state, r.valid_actions(state), [], ep)
  if terminal:
    print(reward)
    break
  ep.append(copy.deepcopy(state))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

0 wmove
[[25. 27. 29. 31. 32. 30. 28. 26.]
 [17. 18. 19. 20. 21. 22. 23. 24.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  2.  3.  4.  5.  6.  7.  8.]
 [ 9. 11. 13. 15. 16. 14. 12. 10.]]

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to ha

In [ ]:
i

46

In [ ]:
r = Representation()
Policy_Network = PolicyNetwork()
import time

s = Simulations()
p,v = Policy_Network(r.state.reshape((1,8,8,97)))
c = time.time()
sp = s.MCTS(r.state,p,v,Policy_Network)
d = time.time()-c
print(d)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

54.32686901092529


In [ ]:
print(np.argmax(sp))

65


In [ ]:
print(np.argmax(p.numpy()[0]))

65


In [ ]:
len(choices)

1000

In [ ]:
import copy
import numpy as np
def st_list(s):
    l = []
    a,b = s.split('1.',maxsplit=1)
    for i in range(2,100):
        try:
            a,b = b.split(str(i)+'.',maxsplit=1)
            l.append(a)
        except:
            l.append(b)
            break

    temp = list(l.pop(-1).split(' '))
    #print(temp)

    bs = []

    for i in l:
        temp_l = i.split(' ')

        bs.append(temp_l[1])
        bs.append(temp_l[2])
    if len(temp)==3:
        x,result = temp[-2],temp[-1]

        bs.append(x)
    else:
        x,y,result = temp[-3],temp[-2],temp[-1]
        #print(x,y,'xy')

        bs.append(x)
        bs.append(y)


    actions = []
    states = []
    current_s = [[25,27,29,31,32,30,28,26],[17,18,19,20,21,22,23,24],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[1,2,3,4,5,6,7,8],[9,11,13,15,16,14,12,10]]
    current_s = np.array(current_s)
    states.append(copy.deepcopy(current_s))
    board_col = {'a':0,'b':1,'c':2,'d':3,'e':4,'f':5,'g':6,'h':7}
    new_el_w = {'Q':[33,34,35,36,37,38,39,40],'R':[41,42,43,44,45,46,47,48],'N':[49,50,51,52,53,54,55,56],'B':[57,58,59,60,61,62,63,64]}
    new_el_b = {'Q':[65,66,67,68,69,70,71,72],'R':[73,74,75,76,77,78,79,80],'N':[81,82,83,84,85,86,87,88],'B':[89,90,91,92,93,94,95,96]}
    for st in range(len(bs)):
        w = bs[st]
        if st%2==0:
            #print(w)
            if w == 'O-O':
                current_s[7][4],current_s[7][5],current_s[7][6],current_s[7][7]=0,10,16,0
            if w == 'O-O-O':
                current_s[7][0],current_s[7][1],current_s[7][2],current_s[7][3],current_s[7][4]=0,0,16,9,0
            if w[0] in "abcdefgh":
                if w[1] in '12345678':
                    #print('b hoo',w[1],w[0],current_s[8-(int(w[1]))+2][board_col[w[0]]])
                    if current_s[8-(int(w[1]))+1][board_col[w[0]]] in [1,2,3,4,5,6,7,8]:
                        current_s[8-(int(w[1]))][board_col[w[0]]]=current_s[8-(int(w[1]))+1][board_col[w[0]]]
                        current_s[8-(int(w[1]))+1][board_col[w[0]]]=0
                    elif current_s[8-(int(w[1]))+2][board_col[w[0]]] in [1,2,3,4,5,6,7,8]:
                        #print('hooo')
                        current_s[8-(int(w[1]))][board_col[w[0]]]=current_s[8-(int(w[1]))+2][board_col[w[0]]]
                        current_s[8-(int(w[1]))+2][board_col[w[0]]]=0
                    if w[1] == '8':
                        if w[2]=='=':
                            current_s[8-(int(w[1]))][board_col[w[0]]]=new_el_w[w[3]].pop(0+current_s[8-(int(w[1]))][board_col[w[0]]]-1)
                        elif w[4]=='=':
                            current_s[8-(int(w[3]))][board_col[w[2]]]=new_el_w[w[5]].pop(0+current_s[8-(int(w[3]))][board_col[w[2]]]-1)
                        else:
                            current_s[8-(int(w[3]))][board_col[w[2]]]=new_el_w[w[4]].pop(0+current_s[8-(int(w[3]))][board_col[w[2]]]-1)
                elif w[1]=='x':
                    if current_s[8-(int(w[3]))][board_col[w[2]]]==0:
                        current_s[8-(int(w[3]))+1][board_col[w[2]]]=0
                    current_s[8-(int(w[3]))][board_col[w[2]]]=current_s[8-(int(w[3]))+1][board_col[w[0]]]
                    current_s[8-(int(w[3]))+1][board_col[w[0]]]=0
                    if w[3]=='8':
                        if w[4]=='=':
                            current_s[8-(int(w[3]))][board_col[w[2]]]=new_el_w[w[5]].pop(0+current_s[8-(int(w[3]))][board_col[w[2]]]-1)
                        else:
                            current_s[8-(int(w[3]))][board_col[w[2]]]=new_el_w[w[4]].pop(0+current_s[8-(int(w[3]))][board_col[w[2]]]-1)
            elif w[0] == 'R':
                if w[1]=='x':
                    w = list(w)
                    w[1],w[2]=w[2],w[3]
                    w = str(''.join(w))
                if w[1] in 'abcdefgh' and w[2] in '12345678':
                    for i in range(1,8):
                        if board_col[w[1]]-i>-1 and current_s[8-int(w[2])][board_col[w[1]]-i] in [9,10]:
                            current_s[8-int(w[2])][board_col[w[1]]]=current_s[8-int(w[2])][board_col[w[1]]-i]
                            current_s[8-int(w[2])][board_col[w[1]]-i]=0
                            break
                        elif board_col[w[1]]-i<0 or current_s[8-int(w[2])][board_col[w[1]]-i]!=0:
                            break
                    for i in range(1,8):
                        if board_col[w[1]]+i<8 and current_s[8-int(w[2])][board_col[w[1]]+i] in [9,10]:
                            current_s[8-int(w[2])][board_col[w[1]]]=current_s[8-int(w[2])][board_col[w[1]]+i]
                            current_s[8-int(w[2])][board_col[w[1]]+i]=0
                            break
                        elif board_col[w[1]]+i>7 or current_s[8-int(w[2])][board_col[w[1]]+i]!=0:
                            break
                    for i in range(1,8):
                        if 8-int(w[2])-i>-1 and current_s[8-int(w[2])-i][board_col[w[1]]] in [9,10]:
                            current_s[8-int(w[2])][board_col[w[1]]]=current_s[8-int(w[2])-i][board_col[w[1]]]
                            current_s[8-int(w[2])-i][board_col[w[1]]]=0
                            break
                        elif 8-int(w[2])-i<0 or current_s[8-int(w[2])-i][board_col[w[1]]]!=0:
                            break
                    for i in range(1,8):
                        if 8-int(w[2])+i<8 and current_s[8-int(w[2])+i][board_col[w[1]]] in [9,10]:
                            current_s[8-int(w[2])][board_col[w[1]]]=current_s[8-int(w[2])+i][board_col[w[1]]]
                            current_s[8-int(w[2])+i][board_col[w[1]]]=0
                            break
                        elif 8-int(w[2])+i>7 or current_s[8-int(w[2])+i][board_col[w[1]]]!=0:
                            break
                elif w[1] in '12345678':
                    if w[2]=='x':
                        w = list(w)
                        w[2],w[3]=w[3],w[4]
                        w = str(''.join(w))
                    current_s[8-int(w[3])][board_col[w[2]]]=current_s[8-int(w[1])][board_col[w[2]]]
                    current_s[8-int(w[1])][board_col[w[2]]]=0
                elif w[1] in 'abcdefgh':
                    if w[2]=='x':
                        w = list(w)
                        w[2],w[3]=w[3],w[4]
                        w = str(''.join(w))
                    try:
                        pos= list(zip(*np.where(current_s==9)))[0]
                        if pos[1]==board_col[w[1]]:
                            current_s[8-(int(w[3]))][board_col[w[2]]]=current_s[pos[0]][pos[1]]
                            current_s[pos[0]][pos[1]]=0
                    except:
                        pass
                    try:
                        pos= list(zip(*np.where(current_s==10)))[0]
                        if pos[1]==board_col[w[1]]:
                            current_s[8-(int(w[3]))][board_col[w[2]]]=current_s[pos[0]][pos[1]]
                            current_s[pos[0]][pos[1]]=0
                    except:
                        pass
            elif w[0] == 'B':
                if w[1]=='x':
                    w = list(w)
                    w[1],w[2]=w[2],w[3]
                    w = str(''.join(w))
                if w[1] in 'abcdefgh':
                    if ( board_col[w[1]]+1+int(w[2]) )%2==0:
                        pos = list(zip(*np.where(current_s==13)))[0]
                        current_s[8-int(w[2])][board_col[w[1]]]=current_s[pos[0]][pos[1]]
                        current_s[pos[0]][pos[1]]=0
                    else:
                        pos = list(zip(*np.where(current_s==14)))[0]
                        current_s[8-int(w[2])][board_col[w[1]]]=current_s[pos[0]][pos[1]]
                        current_s[pos[0]][pos[1]]=0
            elif w[0]=='Q':
                if w[1]=='x':
                    w = list(w)
                    w[1],w[2]=w[2],w[3]
                    w = str(''.join(w))
                if w[1] in 'abcdefgh':
                    for qp in [15,33,34,35,36,37,38,39,40]:
                        try:
                            pos = list(zip(*np.where(current_s==qp)))[0]
                            current_s[8-int(w[2])][board_col[w[1]]]=current_s[pos[0]][pos[1]]
                            current_s[pos[0]][pos[1]]=0
                        except:
                            pass
            elif w[0]=='K':
                if w[1]=='x':
                    w = list(w)
                    w[1],w[2]=w[2],w[3]
                    w = str(''.join(w))
                if w[1] in 'abcdefgh':
                    pos = list(zip(*np.where(current_s==16)))
                    #print(pos)
                    pos = pos[0]
                    current_s[8-int(w[2])][board_col[w[1]]]=current_s[pos[0]][pos[1]]
                    current_s[pos[0]][pos[1]]=0
            elif w[0]=='N':
                if w[1]=='x':
                    w = list(w)
                    w[1],w[2]=w[2],w[3]
                    w = str(''.join(w))
                if w[1] in 'abcdefgh' and w[2] in '12345678':
                    try:
                        pos = list(zip(*np.where(current_s==11)))[0]
                        if (board_col[w[1]]-1 == pos[1] and 8-int(w[2])-2 == pos[0]) or (board_col[w[1]]-2 == pos[1] and 8-int(w[2])-1 == pos[0])or (board_col[w[1]]-2 == pos[1] and 8-int(w[2])+1 == pos[0])or (board_col[w[1]]-1 == pos[1] and 8-int(w[2])+2 == pos[0])or (board_col[w[1]]+1 == pos[1] and 8-int(w[2])+2 == pos[0])or (board_col[w[1]]+2 == pos[1] and 8-int(w[2])+1 == pos[0])or (board_col[w[1]]+2 == pos[1] and 8-int(w[2])-1 == pos[0])or (board_col[w[1]]+1 == pos[1] and 8-int(w[2])-2 == pos[0]):
                            current_s[8-int(w[2])][board_col[w[1]]]=current_s[pos[0]][pos[1]]
                            current_s[pos[0]][pos[1]]=0
                    except:
                        pass
                    try:
                        pos = list(zip(*np.where(current_s==12)))[0]
                        if (board_col[w[1]]-1 == pos[1] and 8-int(w[2])-2 == pos[0]) or (board_col[w[1]]-2 == pos[1] and 8-int(w[2])-1 == pos[0])or (board_col[w[1]]-2 == pos[1] and 8-int(w[2])+1 == pos[0])or (board_col[w[1]]-1 == pos[1] and 8-int(w[2])+2 == pos[0])or (board_col[w[1]]+1 == pos[1] and 8-int(w[2])+2 == pos[0])or (board_col[w[1]]+2 == pos[1] and 8-int(w[2])+1 == pos[0])or (board_col[w[1]]+2 == pos[1] and 8-int(w[2])-1 == pos[0])or (board_col[w[1]]+1 == pos[1] and 8-int(w[2])-2 == pos[0]):
                            current_s[8-int(w[2])][board_col[w[1]]]=current_s[pos[0]][pos[1]]
                            current_s[pos[0]][pos[1]]=0
                    except:
                        pass
                if w[1] in 'abcdefgh' or w[1] in '12345678':
                    if w[2]=='x':
                        w = list(w)
                        w[2],w[3]=w[3],w[4]
                        w = str(''.join(w))
                    try:
                        pos = list(zip(*np.where(current_s==11)))[0]
                        if (w[1] in 'abcdefgh' and pos[1]==board_col[w[1]]) or (w[1] in '12345678' and pos[0]==8-int(w[1]) ):
                            current_s[8-int(w[3])][board_col[w[2]]]=11
                            current_s[pos[0]][pos[1]]=0
                    except:
                        pass
                    try:
                        pos = list(zip(*np.where(current_s==12)))[0]
                        #print(pos,'Ncxd4')
                        if (w[1] in 'abcdefgh' and pos[1]==board_col[w[1]]) or (w[1] in '12345678' and pos[0]==8-int(w[1]) ):
                            #print('in if->w',w)
                            current_s[8-int(w[3])][board_col[w[2]]]=12
                            current_s[pos[0]][pos[1]]=0
                    except:
                        pass
            #print(current_s)
            states.append(copy.deepcopy(current_s))
        else:
            #print(w)
            if w == 'O-O':
                current_s[0][4],current_s[0][5],current_s[0][6],current_s[0][7]=0,26,32,0
            if w == 'O-O-O':
                current_s[0][0],current_s[0][1],current_s[0][2],current_s[0][3],current_s[0][4]=0,0,32,25,0
            if w[0] in "abcdefgh":
                if w[1] in '12345678':
                    if current_s[8-(int(w[1]))-1][board_col[w[0]]] in [17,18,19,20,21,22,23,24]:
                        current_s[8-(int(w[1]))][board_col[w[0]]]=current_s[8-(int(w[1]))-1][board_col[w[0]]]
                        current_s[8-(int(w[1]))-1][board_col[w[0]]]=0
                    elif current_s[8-(int(w[1]))-2][board_col[w[0]]] in [17,18,19,20,21,22,23,24]:
                        current_s[8-(int(w[1]))][board_col[w[0]]]=current_s[8-(int(w[1]))-2][board_col[w[0]]]
                        current_s[8-(int(w[1]))-2][board_col[w[0]]]=0
                    if w[1] == '1':
                        if w[2]=='=':
                            current_s[8-(int(w[1]))][board_col[w[0]]]=new_el_b[w[3]].pop(0+current_s[8-(int(w[1]))][board_col[w[0]]]-17)
                        elif w[4]=='=':
                            current_s[8-(int(w[3]))][board_col[w[2]]]=new_el_b[w[5]].pop(0+current_s[8-(int(w[3]))][board_col[w[2]]]-17)
                        else:
                            current_s[8-(int(w[3]))][board_col[w[2]]]=new_el_b[w[4]].pop(0+current_s[8-(int(w[3]))][board_col[w[2]]]-17)
                elif w[1]=='x':
                    if current_s[8-(int(w[3]))][board_col[w[2]]]==0:
                        current_s[8-(int(w[3]))-1][board_col[w[2]]]=0
                    current_s[8-(int(w[3]))][board_col[w[2]]]=current_s[8-(int(w[3]))-1][board_col[w[0]]]
                    current_s[8-(int(w[3]))-1][board_col[w[0]]]=0
                    if w[3]=='1':
                        if w[4]=='=':
                            current_s[8-(int(w[3]))][board_col[w[2]]]=new_el_b[w[5]].pop(0+current_s[8-(int(w[3]))][board_col[w[2]]]-17)
                        else:
                            current_s[8-(int(w[3]))][board_col[w[2]]]=new_el_b[w[4]].pop(0+current_s[8-(int(w[3]))][board_col[w[2]]]-17)
            elif w[0] == 'R':
                if w[1]=='x':
                    w = list(w)
                    w[1],w[2]=w[2],w[3]
                    w = str(''.join(w))
                if w[1] in 'abcdefgh' and w[2] in '12345678':
                    for i in range(1,8):
                        if board_col[w[1]]-i>-1 and current_s[8-int(w[2])][board_col[w[1]]-i] in [25,26]:
                            current_s[8-int(w[2])][board_col[w[1]]]=current_s[8-int(w[2])][board_col[w[1]]-i]
                            current_s[8-int(w[2])][board_col[w[1]]-i]=0
                            break
                        elif board_col[w[1]]-i<0 or current_s[8-int(w[2])][board_col[w[1]]-i]!=0:
                            break
                    for i in range(1,8):
                        if board_col[w[1]]+i<8 and current_s[8-int(w[2])][board_col[w[1]]+i] in [25,26]:
                            current_s[8-int(w[2])][board_col[w[1]]]=current_s[8-int(w[2])][board_col[w[1]]+i]
                            current_s[8-int(w[2])][board_col[w[1]]+i]=0
                            break
                        elif board_col[w[1]]+i>7 or current_s[8-int(w[2])][board_col[w[1]]+i]!=0:
                            break
                    for i in range(1,8):
                        if 8-int(w[2])-i>-1 and current_s[8-int(w[2])-i][board_col[w[1]]] in [25,26]:
                            current_s[8-int(w[2])][board_col[w[1]]]=current_s[8-int(w[2])-i][board_col[w[1]]]
                            current_s[8-int(w[2])-i][board_col[w[1]]]=0
                            break
                        elif 8-int(w[2])-i<0 or current_s[8-int(w[2])-i][board_col[w[1]]]!=0:
                            break
                    for i in range(1,8):
                        if 8-int(w[2])+i<8 and current_s[8-int(w[2])+i][board_col[w[1]]] in [25,26]:
                            current_s[8-int(w[2])][board_col[w[1]]]=current_s[8-int(w[2])+i][board_col[w[1]]]
                            current_s[8-int(w[2])+i][board_col[w[1]]]=0
                            break
                        elif 8-int(w[2])+i>7 or current_s[8-int(w[2])+i][board_col[w[1]]]!=0:
                            break
                elif w[1] in '12345678':
                    if w[2]=='x':
                          w = list(w)
                          w[2],w[3]=w[3],w[4]
                          w = str(''.join(w))
                    current_s[8-int(w[3])][board_col[w[2]]]=current_s[8-int(w[1])][board_col[w[2]]]
                    current_s[8-int(w[1])][board_col[w[2]]]=0
                elif w[1] in 'abcdefgh':
                    if w[2]=='x':
                        w = list(w)
                        w[2],w[3]=w[3],w[4]
                        w = str(''.join(w))
                    try:
                        pos= list(zip(*np.where(current_s==25)))[0]
                        if pos[1]==board_col[w[1]]:
                            current_s[8-(int(w[3]))][board_col[w[2]]]=current_s[pos[0]][pos[1]]
                            current_s[pos[0]][pos[1]]=0
                    except:
                        pass
                    try:
                        pos= list(zip(*np.where(current_s==26)))[0]
                        if pos[1]==board_col[w[1]]:
                            current_s[8-(int(w[3]))][board_col[w[2]]]=current_s[pos[0]][pos[1]]
                            current_s[pos[0]][pos[1]]=0
                    except:
                        pass
            elif w[0] == 'B':
                if w[1]=='x':
                    w = list(w)
                    w[1],w[2]=w[2],w[3]
                    w = str(''.join(w))
                if w[1] in 'abcdefgh':
                    if ( board_col[w[1]]+1+int(w[2]) )%2==0:
                        pos = list(zip(*np.where(current_s==30)))[0]
                        current_s[8-int(w[2])][board_col[w[1]]]=current_s[pos[0]][pos[1]]
                        current_s[pos[0]][pos[1]]=0
                    else:
                        pos = list(zip(*np.where(current_s==29)))[0]
                        current_s[8-int(w[2])][board_col[w[1]]]=current_s[pos[0]][pos[1]]
                        current_s[pos[0]][pos[1]]=0
            elif w[0]=='Q':
                if w[1]=='x':
                    w = list(w)
                    w[1],w[2]=w[2],w[3]
                    w = str(''.join(w))
                if w[1] in 'abcdefgh':
                    for qp in [31,65,66,67,68,69,70,71,72]:
                        try:
                            pos = list(zip(*np.where(current_s==qp)))[0]
                            current_s[8-int(w[2])][board_col[w[1]]]=current_s[pos[0]][pos[1]]
                            current_s[pos[0]][pos[1]]=0
                        except:
                            pass
            elif w[0]=='K':
                if w[1]=='x':
                    w = list(w)
                    w[1],w[2]=w[2],w[3]
                    w = str(''.join(w))
                if w[1] in 'abcdefgh':
                    pos = list(zip(*np.where(current_s==32)))[0]
                    current_s[8-int(w[2])][board_col[w[1]]]=current_s[pos[0]][pos[1]]
                    current_s[pos[0]][pos[1]]=0
            elif w[0]=='N':
                if w[1]=='x':
                    w = list(w)
                    w[1],w[2]=w[2],w[3]
                    w = str(''.join(w))
                if w[1] in 'abcdefgh':
                    try:
                        pos = list(zip(*np.where(current_s==27)))[0]
                        if (board_col[w[1]]-1 == pos[1] and 8-int(w[2])-2 == pos[0]) or (board_col[w[1]]-2 == pos[1] and 8-int(w[2])-1 == pos[0])or (board_col[w[1]]-2 == pos[1] and 8-int(w[2])+1 == pos[0])or (board_col[w[1]]-1 == pos[1] and 8-int(w[2])+2 == pos[0])or (board_col[w[1]]+1 == pos[1] and 8-int(w[2])+2 == pos[0])or (board_col[w[1]]+2 == pos[1] and 8-int(w[2])+1 == pos[0])or (board_col[w[1]]+2 == pos[1] and 8-int(w[2])-1 == pos[0])or (board_col[w[1]]+1 == pos[1] and 8-int(w[2])-2 == pos[0]):
                            current_s[8-int(w[2])][board_col[w[1]]]=current_s[pos[0]][pos[1]]
                            current_s[pos[0]][pos[1]]=0
                    except:
                        pass
                    try:
                        pos = list(zip(*np.where(current_s==28)))[0]
                        if (board_col[w[1]]-1 == pos[1] and 8-int(w[2])-2 == pos[0]) or (board_col[w[1]]-2 == pos[1] and 8-int(w[2])-1 == pos[0])or (board_col[w[1]]-2 == pos[1] and 8-int(w[2])+1 == pos[0])or (board_col[w[1]]-1 == pos[1] and 8-int(w[2])+2 == pos[0])or (board_col[w[1]]+1 == pos[1] and 8-int(w[2])+2 == pos[0])or (board_col[w[1]]+2 == pos[1] and 8-int(w[2])+1 == pos[0])or (board_col[w[1]]+2 == pos[1] and 8-int(w[2])-1 == pos[0])or (board_col[w[1]]+1 == pos[1] and 8-int(w[2])-2 == pos[0]):
                            current_s[8-int(w[2])][board_col[w[1]]]=current_s[pos[0]][pos[1]]
                            current_s[pos[0]][pos[1]]=0
                    except:
                        pass
                if w[1] in 'abcdefgh' or w[1] in '12345678':
                    if w[2]=='x':
                        w = list(w)
                        w[2],w[3]=w[3],w[4]
                        w = str(''.join(w))
                    try:
                        pos = list(zip(*np.where(current_s==27)))[0]
                        if (w[1] in 'abcdefgh' and pos[1]==board_col[w[1]]) or (w[1] in '12345678' and pos[0]==8-int(w[1]) ):
                            current_s[8-int(w[3])][board_col[w[2]]]=current_s[pos[0]][pos[1]]
                            current_s[pos[0]][pos[1]]=0
                    except:
                        pass
                    try:
                        pos = list(zip(*np.where(current_s==28)))[0]
                        if (w[1] in 'abcdefgh' and pos[1]==board_col[w[1]]) or (w[1] in '12345678' and pos[0]==8-int(w[1]) ):
                            current_s[8-int(w[3])][board_col[w[2]]]=current_s[pos[0]][pos[1]]
                            current_s[pos[0]][pos[1]]=0
                    except:
                        pass
            #print(current_s)
            states.append(copy.deepcopy(current_s))
    return states, result, bs

In [ ]:
def state_action_reward(states, result, moves):
  r = Representation()
  d_states = []
  d_act=[]
  d_actions=[]

  for s in range(len(states)):
    ds = np.zeros((96,8,8))
    for i in range(1,97):
      try:
        pos = list(zip(*np.where(states[s]==i)))[0]
        ds[i-1][pos[0]][pos[1]]=1
      except:
        continue
    if s%2==0:
      ds = np.concatenate((ds,np.ones((1,8,8))))
    else:
      ds = np.concatenate((ds,np.zeros((1,8,8))))
    d_states.append(copy.deepcopy(ds))
  #print(states[1])
  #print(r.actual_state(d_states[1]))
  for m in range(len(moves)):
    mov = moves[m]
    if mov[0] in 'abcdefgh':
      for a in range(104):
        try:
          temp = r.apply_action(d_states[m],a)
          
          if (temp==d_states[m+1]).all():
            d_act.append(a)
            break
        except:
          continue
    elif mov[0]=='R':

      for a in range(104,384):
        try:
          temp = r.apply_action(d_states[m],a)
          
          if (temp==d_states[m+1]).all():
            d_act.append(a)
            break
        except:
          continue
    elif mov[0]=='N':
      for a in range(384,464):
        try:
 
          temp = r.apply_action(d_states[m],a)

          if (temp==d_states[m+1]).all():
            d_act.append(a)
            break
        except:
          continue
    elif mov[0]=='B':
      for a in range(464,744):
        try:
          temp = r.apply_action(d_states[m],a)
          if (temp==d_states[m+1]).all():
            d_act.append(a)
            break
        except:
          continue
    elif mov[0]=='Q':
      for a in range(744,1248):
        try:
          temp = r.apply_action(d_states[m],a)
          if (temp==d_states[m+1]).all():
            d_act.append(a)
            break
        except:
          continue
    else:
      for a in range(1248,1258):
        try:
          temp = r.apply_action(d_states[m],a)
          if (temp==d_states[m+1]).all():
            d_act.append(a)
            break
        except:
          continue
  if result == '1/2-1/2':
    d_reward = np.zeros((len(d_act)))
  elif result == '1-0':
    d_reward = np.zeros((len(d_act)))
    d_reward[::2]=1
    d_reward[1::2]=0
  else:
    d_reward = np.zeros((len(d_act)))
    d_reward[::2]=0
    d_reward[1::2]=1
  for act in d_act:
    temp = np.zeros((1258))
    temp[act]=1
    d_actions.append(copy.deepcopy(temp))

  if len(d_act)== len(states)-1:
    return d_states,d_actions,d_reward
  else:
    return 'error'


In [ ]:
t_f = open('sample_chess_3.txt','r')
t_f = t_f.read()
g_s = t_f.split('\n\n\n')
g_no = 29303
for l in range(1,len(g_s),2):
  g_no+=1
  try:
    g1 = g_s[l]
    g1 = g1.replace('\n',' ')
    g1 = g1.replace('  ',' ')
    s,r,m = st_list(g1)
    st,act,re = state_action_reward(s,r,m)
    for e in range(len(act)):
      data_array = np.array([st[e],act[e],re[e]])
      np.save('/content/ex_games/'+'-G'+str(g_no)+'-'+'-'+ str(e), data_array)
    print(g_no)
  except:
    pass

Streaming output truncated to the last 5000 lines.
24148
24149
24150
24151
24152
24153
24154
24155
24156
24157
24158
24159
24160
24161
24162
24163
24164
24166
24167
24168
24169
24170
24171
24172
24173
24174
24175
24176
24177
24178
24179
24180
24181
24182
24183
24184
24185
24186
24187
24188
24189
24190
24191
24193
24194
24195
24196
24197
24198
24199
24200
24201
24202
24203
24204
24205
24206
24207
24208
24209
24210
24211
24212
24213
24214
24215
24216
24217
24218
24219
24220
24221
24222
24223
24224
24225
24226
24227
24228
24229
24230
24231
24232
24233
24234
24235
24236
24237
24238
24239
24240
24241
24242
24243
24244
24246
24247
24248
24249
24250
24251
24252
24253
24254
24255
24256
24257
24258
24259
24260
24261
24262
24263
24264
24265
24266
24267
24268
24269
24270
24271
24272
24274
24275
24276
24277
24278
24279
24280
24281
24282
24283
24284
24285
24286
24287
24288
24289
24290
24291
24292
24293
24294
24295
24296
24297
24298
24299
24300
24301
24302
24303
24304
24305
24306
24307
24308
24309
2

In [ ]:
len(os.listdir('/content/ex_games'))

163568

In [ ]:
print(s[58])

[[ 0 25  0  0  0 32  0 15]
 [ 0  0 31  0 21  0 26  0]
 [17 18  0 20  4 28  0  0]
 [ 0  0 19  0  0  0  0  0]
 [ 1  0  3  0  0  6  0  0]
 [ 0  0  2  0  5  0  0  8]
 [ 0  0  0 10  0  0  7 13]
 [ 0  0  0  9  0  0 16  0]]


In [ ]:
#ECO "B07"   Rfxc8